# Import Liraries

In [1]:
from __future__ import annotations
import numpy as np
import math
from math import log, ceil
from typing import Dict, Iterable, List, Tuple, Optional, Any, Sequence
from datasets import load_dataset  # pip install datasets
from collections import Counter
from tqdm import tqdm
import inspect


/Users/aymenmejri/Desktop/MyCode/experiments/hdc_v2/hdc_project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Sommaire
1. Architecture générale encoder--decoder HDC
    - M0 : RNG, Clé Rademarcher unique et lot de vecteurs Rademarcher 


# Architecture générale encoder--decoder HDC

## M0 : RNG, Clé Rademarcher unique et lot de vecteurs Rademarcher 

In [2]:
def _rng(seed: int | None) -> np.random.Generator:
    """
    Crée un générateur PCG64 (si seed donné) ou un RNG par défaut (non déterministe).
    """
    return np.random.Generator(np.random.PCG64(seed)) if seed is not None else np.random.default_rng()

In [3]:
def M0_NewKey(seed: int, D: int) -> np.ndarray:
    """
    Génère une clé Rademacher J ∈ {-1,+1}^D (int8), déterministe par seed.
    """
    g = _rng(seed)
    B = g.integers(0, 2, size=D, dtype=np.int8)  # {0,1}
    J = (B << 1) - 1                              # {-1,+1}
    return J

In [4]:
def M0_Rad(n: int, D: int, seed: int | None = None) -> np.ndarray:
    """
    Génère un lot de n vecteurs Rademacher (n, D) en int8. Reproductible si seed fourni.
    """
    g = _rng(seed)
    B = g.integers(0, 2, size=(n, D), dtype=np.int8)
    return (B << 1) - 1

In [5]:
def simhd(X: np.ndarray, Y: np.ndarray) -> float:
    """
    Similarité Hamming signée normalisée: sim(X,Y) = (1/D) <X, Y>.
    Gère X,Y en int8 via upcast en int32 pour la stabilité.
    """
    return float(np.dot(X.astype(np.int32), Y.astype(np.int32)) / X.size)

In [6]:
def test_M0(D: int = 16384, n_pairs: int = 10_000, eps: float = 0.05, seed: int = 123) -> dict:
    """
    Vérifie (HA2) empiriquement:
      - moyenne des simhd(J,J') ≈ 0
      - fréquence empirique P(|sim| > eps) <= 1e-4 (à D=16384, eps=0.05)
      - compare à la borne de Hoeffding 2*exp(-D*eps^2/2)
    """
    g = _rng(seed)
    sims = np.empty(n_pairs, dtype=np.float64)
    for k in range(n_pairs):
        j_seed  = int(g.integers(0, 2**31 - 1))
        jp_seed = int(g.integers(0, 2**31 - 1))
        J  = M0_NewKey(j_seed,  D)
        Jp = M0_NewKey(jp_seed, D)
        sims[k] = simhd(J, Jp)

    mean_sim   = float(sims.mean())
    tail_prob  = float((np.abs(sims) > eps).mean())
    hoeff_bound = 2.0 * math.exp(- D * (eps**2) / 2.0)
    return {
        "D": D, "n_pairs": n_pairs, "eps": eps,
        "mean_sim": mean_sim, "tail_prob": tail_prob, "hoeffding": hoeff_bound
    }

# Exemples d'assertions (à activer dans un vrai test runner):
r = test_M0()
assert abs(r["mean_sim"]) <= 5e-3, f"mean={r['mean_sim']:.4g} trop éloigné de 0"
assert r["tail_prob"] <= 1e-4 + 1e-6, f"queue empirique {r['tail_prob']:.4g} > 1e-4"

In [7]:
def M0_min_D(eps: float, delta: float) -> int:
    """
    Retourne D* tel que 2*exp(-D*eps^2/2) <= delta  =>  D >= 2/eps^2 * log(2/delta).
    """
    assert eps > 0 and 0 < delta < 1, "Paramètres invalides"
    return int(ceil((2.0 / (eps * eps)) * log(2.0 / delta)))

## M1 : Similarité et métrique

Il faudrait voir par la suite si on ne pourrait pas utiliser **simhd** défini précédemment.

In [8]:
def _as_vec(a: np.ndarray) -> np.ndarray:
    a = np.asarray(a)
    if a.ndim != 1:
        raise ValueError(f"attendu un vecteur 1D, shape={a.shape}")
    return a

def M1_sim(X: np.ndarray, Y: np.ndarray) -> float:
    """
    Similarité HDC normalisée: (1/D) <X, Y>.
    - X, Y: vecteurs 1D longueur D, dtypes int8/int32/float32/float64
    - Retour: float64
    """
    X = _as_vec(X); Y = _as_vec(Y)
    if X.shape != Y.shape:
        raise ValueError(f"shapes incompatibles: {X.shape} vs {Y.shape}")
    # cast unique si int8 -> int32, sinon réutilise le buffer
    Xv = X.astype(np.int32, copy=False) if X.dtype == np.int8 else X
    Yv = Y.astype(np.int32, copy=False) if Y.dtype == np.int8 else Y
    D  = Xv.shape[0]
    return float(np.dot(Xv, Yv) / D)

In [9]:
def M1_dH(X: np.ndarray, Y: np.ndarray) -> int:
    """
    Distance de Hamming (nombre de coordonnées différentes).
    Si X,Y ∈ {-1,+1}^D, identité: dH = D/2 * (1 - sim).
    """
    X = _as_vec(X); Y = _as_vec(Y)
    if X.shape != Y.shape:
        raise ValueError(f"shapes incompatibles: {X.shape} vs {Y.shape}")
    D  = X.shape[0]
    sim = M1_sim(X, Y)
    # L'identité produit un entier pour {-1,+1}; on arrondit prudemment.
    return int(round((D/2.0) * (1.0 - sim)))

In [10]:
def _as_mat(A: np.ndarray) -> np.ndarray:
    A = np.asarray(A)
    if A.ndim != 2:
        raise ValueError(f"attendu une matrice 2D, shape={A.shape}")
    return A

def M1_sim_batch(Xs: np.ndarray, Ys: np.ndarray) -> np.ndarray:
    """
    Similarités ligne-à-ligne pour deux matrices (n,D):
      sim_k = (1/D) <Xs[k], Ys[k]>   pour k=0..n-1
    Retour: (n,) en float64.
    """
    Xs = _as_mat(Xs); Ys = _as_mat(Ys)
    if Xs.shape != Ys.shape:
        raise ValueError(f"shapes incompatibles: {Xs.shape} vs {Ys.shape}")
    D  = Xs.shape[1]
    Xv = Xs.astype(np.int32, copy=False) if Xs.dtype == np.int8 else Xs
    Yv = Ys.astype(np.int32, copy=False) if Ys.dtype == np.int8 else Ys
    dots = (Xv * Yv).sum(axis=1, dtype=np.int64)   # somme sûre
    return (dots / D).astype(np.float64, copy=False)

In [11]:
def _rand_pm1(n: int, D: int, seed: int) -> np.ndarray:
    g = np.random.default_rng(seed)
    B = g.integers(0, 2, size=(n, D), dtype=np.int8)  # {0,1}
    return (B << 1) - 1                                # {-1,+1}

def test_M1_identity(D: int = 4096, n: int = 1000, seed: int = 7) -> None:
    X = _rand_pm1(n, D, seed)
    Y = _rand_pm1(n, D, seed + 1)
    sim = M1_sim_batch(X, Y)
    dH  = (D/2.0 * (1.0 - sim)).astype(int)
    # Comptage direct des désaccords
    mis = ((X * Y) == -1).sum(axis=1)
    assert np.all(dH == mis), "identité Hamming <-> produit violée"

    # Bords (quelques échantillons)
    for k in range(3):
        xk = X[k]
        assert M1_sim(xk, xk) == 1.0
        assert M1_dH(xk, xk)  == 0
        assert M1_sim(xk, -xk) == -1.0
        assert M1_dH(xk, -xk)  == D

def test_M1_batch_equivalence(D: int = 8192, n: int = 200, seed: int = 11) -> None:
    X = _rand_pm1(n, D, seed)
    Y = _rand_pm1(n, D, seed + 1)
    sb = M1_sim_batch(X, Y)
    # version naïve (boucle) pour contrôle
    sn = np.array([M1_sim(X[k], Y[k]) for k in range(n)], dtype=np.float64)
    assert np.allclose(sb, sn, rtol=0, atol=1e-12), "écart batch vs naïf"

In [12]:
def test_M1_dtypes_and_var(D: int = 16384, n: int = 2000, seed: int = 21) -> None:
    X = _rand_pm1(n, D, seed).astype(np.int8)
    Y = _rand_pm1(n, D, seed + 1).astype(np.int8)

    # dtypes flottants
    Xf32, Yf32 = X.astype(np.float32), Y.astype(np.float32)
    Xf64, Yf64 = X.astype(np.float64), Y.astype(np.float64)

    s_int  = M1_sim_batch(X, Y)
    s_f32  = M1_sim_batch(Xf32, Yf32)
    s_f64  = M1_sim_batch(Xf64, Yf64)
    assert np.allclose(s_int, s_f32, rtol=0, atol=1e-12)
    assert np.allclose(s_int, s_f64, rtol=0, atol=1e-12)

    # variance ≈ 1/D sous indépendance
    var_emp = float(s_int.var(ddof=1))
    assert abs(var_emp - 1.0/D) < 5.0/D, f"var={var_emp:.3g} vs 1/D"

In [13]:
test_M1_identity()
test_M1_batch_equivalence()
test_M1_dtypes_and_var()

## M2 . Permutation positionnelle

In [14]:
def M2_roll(X: np.ndarray, Delta: int) -> np.ndarray:
    """
    Rotation circulaire (décalage modulo D).
    - X: vecteur 1D (D,) ou matrice (n,D)
    - Delta: entier (peut être négatif)
    - Retour: vue recopiée (np.roll) même dtype (ex. int8)
    """
    A = np.asarray(X)
    if A.ndim == 1:
        return np.roll(A, shift=Delta)
    if A.ndim == 2:
        return np.roll(A, shift=Delta, axis=1)
    raise ValueError(f"shape non supportée: {A.shape}")

In [15]:
def M2_plan_perm(D: int, seed: int | None = None) -> np.ndarray:
    """
    Crée une permutation bijective π sur {0..D-1}.
    """
    g  = np.random.default_rng(seed)
    pi = g.permutation(D)                  # π : indices cibles
    # Vérif bijectivité
    if np.unique(pi).size != D:
        raise RuntimeError("π non bijective")
    return pi.astype(np.int64, copy=False)

In [16]:
def _cycles_of(pi: np.ndarray) -> tuple[np.ndarray, np.ndarray, list[np.ndarray]]:
    """
    Retourne (cycle_id, pos_in_cycle, cycles) pour π.
    - cycle_id[i]   : identifiant de cycle du point i
    - pos_in_cycle[i]: position de i dans son cycle
    - cycles        : liste de tableaux numpy des indices de chaque cycle
    """
    D = pi.size
    seen = np.zeros(D, dtype=bool)
    cycles: list[np.ndarray] = []
    cycle_id = -np.ones(D, dtype=np.int64)
    pos      = -np.ones(D, dtype=np.int64)
    cid = 0
    for i in range(D):
        if not seen[i]:
            cur = []
            j = i
            while not seen[j]:
                seen[j] = True
                cur.append(j)
                j = int(pi[j])
            cyc = np.array(cur, dtype=np.int64)
            for p, idx in enumerate(cyc):
                cycle_id[idx] = cid
                pos[idx]      = p
            cycles.append(cyc); cid += 1
    return cycle_id, pos, cycles

def M2_pow_index(pi: np.ndarray, k: int) -> np.ndarray:
    """
    Calcule l'index idx tel que X[idx] = Π^k_pi(X).
    """
    cycle_id, pos, cycles = _cycles_of(pi)
    D   = pi.size
    idx = np.empty(D, dtype=np.int64)
    for cyc in cycles:
        L = cyc.size
        # décalage modulo L
        tgt = np.roll(cyc, shift=k % L)
        # mapping: cyc[p] -> tgt[p]
        idx[cyc] = tgt
    return idx

In [17]:
def M2_perm_pow(X: np.ndarray, pi: np.ndarray, k: int) -> np.ndarray:
    """
    Applique Π^k_pi à X (1D ou 2D). Réutilise un index idx = π^k.
    """
    A = np.asarray(X)
    idx = M2_pow_index(pi, k)
    if A.ndim == 1:
        return A[idx]
    if A.ndim == 2:
        return A[:, idx]
    raise ValueError(f"shape non supportée: {A.shape}")

In [18]:
def _rand_pm1(n: int, D: int, seed: int) -> np.ndarray:
    g = np.random.default_rng(seed)
    B = g.integers(0, 2, size=(n, D), dtype=np.int8)
    return (B << 1) - 1  # {-1,+1}

def test_M2_isometry_invertibility(D: int = 4096, n: int = 1000, seed: int = 7) -> None:
    g   = np.random.default_rng(seed)
    pi  = M2_plan_perm(D, seed=seed)
    X   = _rand_pm1(n, D, seed+1)
    Y   = _rand_pm1(n, D, seed+2)
    ks  = g.integers(-5*D, 5*D, size=10)  # k variés (mod L des cycles)
    for k in ks:
        Xp = M2_perm_pow(X, pi, int(k))
        Yp = M2_perm_pow(Y, pi, int(k))
        # Isométrie (échantillons)
        for t in range(5):
            s1 = M1_sim(X[t], Y[t])
            s2 = M1_sim(Xp[t], Yp[t])
            assert s1 == s2, "isométrie violée"
        # Inversibilité
        Xback = M2_perm_pow(Xp, pi, int(-k))
        assert np.array_equal(X, Xback), "inverse incorrect"
        # Distribution (comptages ±1)
        cnt = (X[0] == 1).sum()
        cntp = (Xp[0] == 1).sum()
        assert cnt == cntp, "distribution ±1 non conservée"

def test_M2_roll_isometry(D: int = 4096, n: int = 200, seed: int = 11) -> None:
    X = _rand_pm1(n, D, seed)
    Y = _rand_pm1(n, D, seed+1)
    for Delta in (-17, -1, 0, 1, 37):
        Xr = M2_roll(X, Delta)
        Yr = M2_roll(Y, Delta)
        # vérifier quelques paires
        for t in range(3):
            assert M1_sim(X[t], Y[t]) == M1_sim(Xr[t], Yr[t])
        # inversibilité
        assert np.array_equal(M2_roll(M2_roll(X, Delta), -Delta), X)

In [19]:
test_M2_isometry_invertibility()
test_M2_roll_isometry()

## M3 . Binding (Hadamard / XNOR)

In [20]:
def _ensure_pm1_int8(A: np.ndarray) -> np.ndarray:
    """Vérifie que A ne contient que {-1,+1} et est en int8."""
    B = np.asarray(A)
    if B.dtype != np.int8:
        B = B.astype(np.int8, copy=False)
    # (option) assertions rapides: 
    # if not np.all((B == -1) | (B == 1)): raise ValueError("non ±1")
    return B

def M3_bind(X: np.ndarray, J: np.ndarray) -> np.ndarray:
    """
    Binding Hadamard: (X * J) en conservant le dtype int8.
    - X, J: (D,) ou (n,D) avec broadcast sur J (D,)
    - Retour: même shape que X, dtype int8
    """
    Xv = _ensure_pm1_int8(X)
    Jv = _ensure_pm1_int8(J)
    return (Xv * Jv).astype(np.int8, copy=False)

def M3_unbind(Y: np.ndarray, J: np.ndarray) -> np.ndarray:
    """
    Unbinding = binding avec la même clé (involutif).
    """
    Yv = _ensure_pm1_int8(Y)
    Jv = _ensure_pm1_int8(J)
    return (Yv * Jv).astype(np.int8, copy=False)

In [21]:
def _rand_pm1(n: int, D: int, seed: int) -> np.ndarray:
    g = np.random.default_rng(seed)
    B = g.integers(0, 2, size=(n, D), dtype=np.int8)
    return (B << 1) - 1

def test_M3_isometry_involution(D: int = 4096, n: int = 2000, seed: int = 21) -> None:
    from math import isclose
    # échantillons
    X = _rand_pm1(n, D, seed)
    Y = _rand_pm1(n, D, seed+1)
    J = _rand_pm1(1, D, seed+2)[0]  # clé (D,)
    # isométrie (quelques paires)
    for t in range(10):
        s1 = M1_sim(X[t], Y[t])
        Xb, Yb = M3_bind(X[t], J), M3_bind(Y[t], J)
        s2 = M1_sim(Xb, Yb)
        assert s1 == s2, "isométrie violée"
        # involutivité
        assert np.array_equal(M3_unbind(Xb, J), X[t]), "involutivité violée"

In [22]:
test_M3_isometry_involution()

In [23]:
def M3_bind_batch(X: np.ndarray, J: np.ndarray) -> np.ndarray:
    """
    Binding Hadamard par lot (batch).
    
    Paramètres
    ----------
    X : np.ndarray
        Tableau de forme (n, D) en {-1, +1}, dtype=int8 recommandé.
    J : np.ndarray
        Clé unique de forme (D,) en {-1, +1}, dtype=int8.
    
    Retour
    ------
    np.ndarray
        Résultat de forme (n, D), dtype=int8.
    """
    Xv = _ensure_pm1_int8(X)
    Jv = _ensure_pm1_int8(J)

    if Xv.ndim != 2:
        raise ValueError(f"X doit être de rang 2 (n,D), reçu shape={Xv.shape}")
    if Jv.ndim != 1:
        raise ValueError(f"J doit être un vecteur (D,), reçu shape={Jv.shape}")
    if Xv.shape[1] != Jv.size:
        raise ValueError(f"Dimensions incompatibles: X{Xv.shape}, J{Jv.shape}")

    return (Xv * Jv).astype(np.int8, copy=False)

def test_M3_cross_tranche(D: int = 16384, n: int = 4000, eps: float = 0.05, seed: int = 33) -> dict:
    X = _rand_pm1(n, D, seed)
    Y = _rand_pm1(n, D, seed+1)
    J  = _rand_pm1(1, D, seed+2)[0]
    Jp = _rand_pm1(1, D, seed+3)[0]
    Xb, Ybp = M3_bind_batch(X, J), M3_bind_batch(Y, Jp)
    sims = M1_sim_batch(Xb, Ybp)  # M1 (batch)
    mean_sim  = float(sims.mean())
    tail_prob = float((np.abs(sims) > eps).mean())
    bound     = 2.0 * math.exp(- D * eps * eps / 2.0)
    # petits checks
    assert abs(mean_sim) < 5e-3 + 5e-3, "centrage éloigné de 0"
    return {"mean": mean_sim, "tail": tail_prob, "hoeffding": bound}

In [24]:
test_M3_cross_tranche()

{'mean': -1.373291015625e-06, 'tail': 0.0, 'hoeffding': 2.5508152590520792e-09}

In [25]:
def M3_xnor_bind_bits(xbits: np.ndarray, jbits: np.ndarray) -> np.ndarray:
    """
    xbits, jbits: tableaux d'octets (packés) représentant {0,1}.
    Retour: xnor(xbits, jbits) packé (même shape), sans remap.
    (Implémentation complète et remap -> ±1 seront intégrés au module d'optimisation.)
    """
    if xbits.dtype != np.uint8 or jbits.dtype != np.uint8:
        raise ValueError("bits attendus en uint8")
    if xbits.shape != jbits.shape:
        raise ValueError("shapes incompatibles")
    # XNOR = NOT XOR
    return np.bitwise_not(np.bitwise_xor(xbits, jbits))

## M4 . Lexique EN (hypervecteurs types)

In [26]:

def _canon_token(w: str) -> str:
    """Normalisation légère: trim + lower; extensible selon besoin."""
    return w.strip().lower()

class M4_LexEN:
    """
    Lexique EN: w -> L_en(w) ∈ {-1,+1}^D (int8, readonly).
    - RNG PCG64 dédié pour reproductibilité.
    - Option: pool OOV pré-généré pour réduire la latence.
    - Persistance: save/load .npz (portable).
    """
    def __init__(self, seed: int, D: int, reserve_pool: int = 0):
        self.D: int = int(D)
        self.rng = np.random.default_rng(seed)
        self.table: Dict[str, np.ndarray] = {}
        self._reserve_pool: List[np.ndarray] = []
        for _ in range(int(reserve_pool)):
            v = self._new_key()
            v.setflags(write=False)
            self._reserve_pool.append(v)
        self._pool_idx: int = 0

    def _new_key(self) -> np.ndarray:
        bits = self.rng.integers(0, 2, size=self.D, dtype=np.int8)
        return ((bits << 1) - 1).astype(np.int8, copy=False)

    def get(self, w: str, use_pool: bool = False) -> np.ndarray:
        key = _canon_token(w)
        v = self.table.get(key, None)
        if v is not None:
            return v
        if use_pool and self._reserve_pool:
            v = self._reserve_pool[self._pool_idx]
            self._pool_idx = (self._pool_idx + 1) % len(self._reserve_pool)
        else:
            v = self._new_key()
            v.setflags(write=False)       # immutabilité côté appelant
        self.table[key] = v
        return v

    def get_many(self, words: Iterable[str], use_pool: bool = False) -> np.ndarray:
        """
        Retourne une matrice (n, D) int8 alignée en mémoire (C-contiguë).
        """
        arr = np.empty((len(list(words)), self.D), dtype=np.int8)
        # NB: on boucle pour éviter des copies implicites; n ~ phrase courte => OK.
        for i, w in enumerate(words):
            arr[i, :] = self.get(w, use_pool=use_pool)
        return arr

    def contains(self, w: str) -> bool:
        return _canon_token(w) in self.table

    def size(self) -> int:
        return len(self.table)

    def save(self, path: str) -> None:
        keys = np.array(list(self.table.keys()), dtype=object)
        mats = np.stack([self.table[k] for k in keys], axis=0).astype(np.int8, copy=False)
        np.savez_compressed(path, D=self.D, keys=keys, mats=mats)

    @staticmethod
    def load(path: str) -> "M4_LexEN":
        data = np.load(path, allow_pickle=True)
        D = int(data["D"])
        keys: np.ndarray = data["keys"]
        mats: np.ndarray = data["mats"].astype(np.int8, copy=False)
        # seed fictif (non utilisé pour les entrées déjà présentes)
        lex = M4_LexEN(seed=0, D=D)
        for k, v in zip(keys.tolist(), mats):
            vv = v.view()
            vv.setflags(write=False)
            lex.table[k] = vv
        return lex

# Raccourcis API:
def M4_LexEN_new(seed: int, D: int, reserve_pool: int = 0) -> M4_LexEN:
    return M4_LexEN(seed, D, reserve_pool)

def M4_get(Lex: M4_LexEN, w: str, use_pool: bool = False) -> np.ndarray:
    return Lex.get(w, use_pool=use_pool)

def M4_get_many(Lex: M4_LexEN, words: Iterable[str], use_pool: bool = False) -> np.ndarray:
    return Lex.get_many(words, use_pool=use_pool)

In [27]:
def test_M4_seed_stability(D: int = 16384, seed: int = 123) -> None:
    L1 = M4_LexEN_new(seed, D)
    L2 = M4_LexEN_new(seed, D)
    v1 = M4_get(L1, "Cat")
    v2 = M4_get(L2, "cat")  # même token après normalisation
    assert np.array_equal(v1, v2), "instables malgré seed identique"
    # immutabilité:
    try:
        v1[0] = 0
        assert False, "le vecteur devrait être readonly"
    except ValueError:
        pass

In [28]:
test_M4_seed_stability()

In [29]:
def M4_collision_audit(Lex: M4_LexEN, vocab: List[str]) -> float:
    mats = np.stack([Lex.get(w) for w in vocab], axis=0).astype(np.int16, copy=False)
    D = mats.shape[1]
    # Gram normalisé
    G = (mats @ mats.T) / D
    np.fill_diagonal(G, 0.0)
    return float(np.max(np.abs(G)))



In [30]:
def test_M4_collisions(D: int = 16384, V: int = 2000, seed: int = 321) -> None:
    Lex = M4_LexEN_new(seed, D)
    vocab = [f"w{i}" for i in range(V)]
    m = M4_collision_audit(Lex, vocab)
    assert m <= 0.05 + 1e-6, f"max |sim|={m:.3f} > 0.05 (D trop petit?)"
    
def test_M4_oov_pool(D: int = 8192, seed: int = 77, R: int = 512, N: int = 400) -> None:
    Lex = M4_LexEN_new(seed, D, reserve_pool=R)
    seen = set()
    for i in range(N):
        v = M4_get(Lex, f"OOV-{i}", use_pool=True)
        seen.add(id(v))
    reuse_rate = 1.0 - len(seen) / N  # part de réutilisations (si N>R)
    assert reuse_rate <= max(0.0, (N - R) / N + 1e-9)

def test_M4_persist(tmp_path: Optional[str] = None) -> None:
    import os, tempfile
    if tmp_path is None:
        tmp_path = tempfile.mkdtemp()
    path = os.path.join(tmp_path, "lex.npz")
    Lex = M4_LexEN_new(42, 4096)
    ref = [M4_get(Lex, w) for w in ["a", "b", "c"]]
    Lex.save(path)
    Lex2 = M4_LexEN.load(path)
    out = [M4_get(Lex2, w) for w in ["a", "b", "c"]]
    for r, o in zip(ref, out):
        assert np.array_equal(r, o)

In [31]:
test_M4_collisions()
test_M4_oov_pool()
test_M4_persist()

In [32]:
def M4_pair_stats(Lex: M4_LexEN, vocab: List[str], n_pairs: int = 50_000, seed: int = 5) -> Tuple[float, float]:
    g = np.random.default_rng(seed)
    V = len(vocab)
    sims = np.empty(n_pairs, dtype=np.float64)
    for k in range(n_pairs):
        i, j = g.integers(0, V), g.integers(0, V)
        while j == i:
            j = g.integers(0, V)
        Xi, Xj = Lex.get(vocab[i]).astype(np.int32, copy=False), Lex.get(vocab[j]).astype(np.int32, copy=False)
        sims[k] = (Xi @ Xj) / Lex.D
    return float(sims.mean()), float(sims.var())

## M5 . Compositeur de n-grammes

In [33]:
def sign_int(x: np.ndarray) -> np.ndarray:
    """
    Retourne un vecteur int8 dans {-1,+1} à partir d'un tableau entier x.
    Convention: 0 -> +1 (rare si n est impair).
    """
    y = (x >= 0).astype(np.int8, copy=False)
    return ((y << 1) - 1).astype(np.int8, copy=False)

In [34]:
def M5_precompute_pi_pows(pi: np.ndarray, n: int) -> list[np.ndarray]:
    """
    Pré-calcul des indices pour Pi^j, j=0..n-1. 
    pi est une permutation (shape (D,)).
    """
    D = pi.shape[0]
    idxs = [np.arange(D)]
    for j in range(1, n):
        idxs.append(pi[idxs[-1]])
    # rendre en lecture seule (sécurité)
    for a in idxs: a.setflags(write=False)
    return idxs

In [35]:
def M5_ngram_cached(LexEN, pi_pows: List[np.ndarray], tokens: List[str]) -> np.ndarray:
    """
    E_t = sign( sum_{j=0..n-1} Pi^j L(w_{t-j}) ), 
    avec pi_pows[j] = indices pour Pi^j.
    """
    n = len(tokens); D = LexEN.D
    acc = np.zeros(D, dtype=np.int16)
    # j=0 correspond au token courant; tokens = [w_{t-n+1},...,w_t]
    for j, w in enumerate(tokens[::-1]):
        Lw = LexEN.get(w).astype(np.int16, copy=False)
        acc += Lw[pi_pows[j]]
    return sign_int(acc)

In [36]:
def M5_ngram(LexEN, pi: np.ndarray, tokens: List[str]) -> np.ndarray:
    n = len(tokens); D = LexEN.D
    acc = np.zeros(D, dtype=np.int16)
    base = np.arange(D)
    # pour éviter de refaire base à chaque j, on itère sur un idx courant
    idx = base
    # on veut Pi^j pour j = 0..n-1, appliqué au token w_{t-j}
    # tokens[::-1] correspond à j=0..n-1
    for j, w in enumerate(tokens[::-1]):
        if j == 0:
            idx = base  # Pi^0
        else:
            idx = pi[idx]  # composition
        Lw = LexEN.get(w).astype(np.int16, copy=False)
        acc += Lw[idx]
    return sign_int(acc)

In [37]:
def M5_window_stream(LexEN, pi: np.ndarray, words: List[str], n: int) -> List[np.ndarray]:
    """
    Renvoie la liste [E_n, E_{n+1}, ..., E_T].
    Pré-calcule une seule fois les Pi^j.
    """
    pi_pows = M5_precompute_pi_pows(pi, n)
    out: List[np.ndarray] = []
    for t in range(n-1, len(words)):
        window = words[max(0, t-n+1): t+1]  # longueur n (supposer len >= n)
        out.append(M5_ngram_cached(LexEN, pi_pows, window))
    return out

In [38]:
def _rand_word(g, V): return f"w{int(g.integers(0, V))}"

def test_M5_separability(D=16384, n=3, V=2000, trials=500, seed=77):
    """
    Correction: fam1b est construit en perturbant (au plus) 1 token d'une
    même base de n-grammes, plutôt que de tirer des tokens indépendants.
    Attendu: intra > 0.2 ; inter ≈ 0.
    """
    g = np.random.default_rng(seed)

    # Dépendances (cohérentes avec vos modules)
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)
    pi_pows = M5_precompute_pi_pows(pi, n)

    # 1) Génère les séquences de base (trials x n)
    base_seqs = [[_rand_word(g, V) for _ in range(n)] for _ in range(trials)]

    # 2) fam1 = n-grammes exacts issus des bases
    fam1 = [M5_ngram_cached(Lex, pi_pows, toks) for toks in base_seqs]

    # 3) fam1b = copies perturbées (au plus 1 token remplacé avec prob 0.2)
    fam1b = []
    for toks in base_seqs:
        toks2 = toks.copy()
        if g.random() < 0.2:  # petite perturbation de structure
            j = int(g.integers(0, n))
            toks2[j] = _rand_word(g, V)
        fam1b.append(M5_ngram_cached(Lex, pi_pows, toks2))

    # 4) fam2 = autre famille indépendante (n-grammes disjoints en moyenne)
    fam2 = [M5_ngram_cached(Lex, pi_pows, [_rand_word(g, V) for _ in range(n)]) 
            for _ in range(trials)]

    # Mesures (vectorisées)
    A = np.stack(fam1).astype(np.int16, copy=False)   # (trials,D)
    B = np.stack(fam1b).astype(np.int16, copy=False)  # (trials,D)
    C = np.stack(fam2).astype(np.int16, copy=False)   # (trials,D)

    # Similarité "intra" paire-à-paire (i,i) entre fam1 et fam1b
    intra = float(np.mean(np.sum(A * B, axis=1) / D))

    # Similarité "inter" moyenne entre fam1 et fam2 (toutes paires)
    # NB: on peut prendre l'absolu, la moyenne simple doit être centrée ~0
    inter = float(np.mean(np.abs((A @ C.T) / D)))

    assert intra > 0.2 - 1e-3, f"intra {intra:.3f} trop faible"
    assert inter < 0.05 + 1e-3, f"inter {inter:.3f} trop élevé"

In [39]:
def test_M5_drop_robustness(D=16384, n_list=(2,3,5), V=2000, trials=200, drop=0.1, seed=99):
    g = np.random.default_rng(seed)
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)
    results = []
    for n in n_list:
        pi_pows = M5_precompute_pi_pows(pi, n)
        fam = [[_rand_word(g,V) for _ in range(n)] for _ in range(trials)]
        E_ref = [M5_ngram_cached(Lex, pi_pows, toks) for toks in fam]
        # drop: remplacer chaque token avec prob drop
        def apply_drop(toks):
            return [tok if g.random() > drop else _rand_word(g,V) for tok in toks]
        E_drop = [M5_ngram_cached(Lex, pi_pows, apply_drop(t)) for t in fam]
        A = np.stack(E_ref).astype(np.int16)
        B = np.stack(E_drop).astype(np.int16)
        sim = float(np.mean(np.sum(A*B, axis=1) / D))
        results.append((n, sim))
    # On s'attend à une décroissance de sim quand n augmente (plus de contraintes).
    for k in range(1, len(results)):
        assert results[k][1] <= results[k-1][1] + 1e-3

def test_M5_edges(D=4096, n=3, seed=5):
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)
    pi_pows = M5_precompute_pi_pows(pi, n)
    toks = ["a","b","c"][:n]
    E = M5_ngram_cached(Lex, pi_pows, toks)
    # bords M1
    assert M1_sim(E, E) == 1.0
    assert M1_dH(E, E) == 0
    # isométrie M2
    Epi = E[pi]  # Pi^1 pour test
    assert M1_sim(Epi, Epi) == 1.0

In [40]:
test_M5_separability()
test_M5_drop_robustness()
test_M5_edges()

## M6 . Accumulateur de segment & seuillage (majorité)

In [41]:
def M6_SegAcc_init(D: int) -> np.ndarray:
    """
    Initialise l'accumulateur de segment en int16 (zéro).
    """
    return np.zeros(int(D), dtype=np.int16)

In [42]:
def M6_SegAcc_pushPos(S: np.ndarray, X_t: np.ndarray, K_s: np.ndarray) -> np.ndarray:
    """
    Ajoute (X_t ⊗ K_s) à l'accumulateur S.
    X_t,K_s sont int8 dans {-1,+1}. S est int16.
    """
    # binding hadamard (M3), upcast en int16 pour l'addition
    S += (X_t.astype(np.int16, copy=False) * K_s.astype(np.int16, copy=False))
    return S

In [43]:
def M6_SegAcc_pushE(S: np.ndarray, E_t: np.ndarray, Delta: int,
                    K_s: np.ndarray, pi_pows: List[np.ndarray]) -> np.ndarray:
    """
    Ajoute (Pi^Delta E_t ⊗ K_s) à S.
    E_t,K_s int8 {-1,+1}; S int16; pi_pows[j] = indices pour Pi^j.
    """
    idx = pi_pows[Delta]  # suppose Delta < len(pi_pows)
    # re-indexation + binding + accumulation
    S += (E_t[idx].astype(np.int16, copy=False) * K_s.astype(np.int16, copy=False))
    return S

In [44]:
def M6_SegAcc_sign(S: np.ndarray) -> np.ndarray:
    """
    Retourne H = sign_strict(S) en int8 dans {-1,+1}.
    Règle stricte : +1 si S>0, sinon -1.
    """
    return np.where(S > 0, 1, -1).astype(np.int8, copy=False)

In [45]:
def test_M6_basic(D=4096, seed=12):
    g = np.random.default_rng(seed)
    X = ((g.integers(0,2,size=D,dtype=np.int8) << 1) - 1)
    Y = ((g.integers(0,2,size=D,dtype=np.int8) << 1) - 1)
    K = ((g.integers(0,2,size=D,dtype=np.int8) << 1) - 1)
    # involutivité
    assert np.array_equal((X*K)*K, X)
    # isométrie
    from math import isclose
    dot_xy = int(np.dot(X.astype(np.int32), Y.astype(np.int32)))
    dot_b  = int(np.dot((X*K).astype(np.int32), (Y*K).astype(np.int32)))
    assert dot_xy == dot_b

In [46]:
test_M6_basic()

In [47]:
def kl_half_vs_p(p: float) -> float:
    """
    KL(1/2 || p) = (1/2) * [log(1/2p) + log(1/2(1-p))] = 0.5*log(1/(4 p (1-p))).
    C'est le taux de Chernoff exact pour le seuil 1/2 sur des Bernoulli(p).
    """
    if not (0.0 < p < 1.0):
        raise ValueError("p doit être dans (0,1)")
    return 0.5 * math.log(1.0 / (4.0 * p * (1.0 - p)))


def M6_simulate_majority_error_fast(
    D: int = 8192,
    m_list=(8, 16, 32, 64, 128),
    p: float = 0.55,
    trials: int = 5000,
    seed: int = 33,
    strict_tie: bool = True,
    batch_trials: int = 2048,
):
    """
    Simulation optimisée: on simule #(+1) ~ Binomial(m,p) pour (trials, D).
    'strict_tie=True' -> majority stricte (> m/2), sinon non stricte (>= m/2).
    """
    g = np.random.default_rng(seed)
    out = []
    for m in m_list:
        thr = (m // 2) + 1 if strict_tie else (m + 1) // 2
        total_ok = 0
        done = 0
        while done < trials:
            b = min(batch_trials, trials - done)
            cnt = g.binomial(n=m, p=p, size=(b, D)).astype(np.int16, copy=False)
            total_ok += int((cnt >= thr).sum())
            done += b
        err_emp = 1.0 - (total_ok / float(trials * D))
        # borne simple (quadratique) donnée à titre indicatif
        bound_hoeff = math.exp(-0.5 * m * (2 * p - 1) ** 2)
        out.append((m, float(err_emp), float(bound_hoeff)))
    return out


def M6_fit_loglinear(points, m_min_for_fit: int = 32):
    """
    Fit log(err_emp) = a + b*m sur m >= m_min_for_fit.
    Retourne {intercept, slope, R2}.
    """
    arr = np.array(points, dtype=np.float64)    # colonnes: m, err_emp, bound
    m_all = arr[:, 0]
    y_all = np.log(np.maximum(arr[:, 1], 1e-14))  # plancher num.
    mask = (m_all >= m_min_for_fit)
    m = m_all[mask]
    y = y_all[mask]
    A = np.vstack([np.ones_like(m), m]).T
    coef, *_ = np.linalg.lstsq(A, y, rcond=None)
    yhat = A @ coef
    ss_res = float(np.sum((y - yhat) ** 2))
    ss_tot = float(np.sum((y - y.mean()) ** 2))
    R2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 1.0
    return dict(intercept=float(coef[0]), slope=float(coef[1]), R2=R2)


def test_M6_majority_acceptance(
    D: int = 8192,
    p: float = 0.60,
    m_list=(8, 16, 32, 64, 128, 192),
    trials: int = 8000,
    seed: int = 44,
    strict_tie: bool = True,
    m_min_for_fit: int = 64,
    rel_tol_vs_KL: float = 0.25  # 25% de tolérance vs pente KL
):
    """
    Test d'acceptation ajusté:
      - Simulation rapide (binomiale).
      - Fit sur m >= m_min_for_fit (réduit la courbure aux petits m).
      - Comparaison à la pente théorique -KL(1/2||p) (plus fidèle que Hoeffding).
    """
    pts = M6_simulate_majority_error_fast(
        D=D, m_list=m_list, p=p, trials=trials, seed=seed,
        strict_tie=strict_tie, batch_trials=2048
    )
    fit = M6_fit_loglinear(pts, m_min_for_fit=m_min_for_fit)
    slope_emp = fit["slope"]
    slope_th  = - kl_half_vs_p(p)    # pente attendue en asymptote
    rel_gap   = abs(slope_emp - slope_th) / abs(slope_th)

    # critères: linéarité forte + pente raisonnablement proche du taux KL
    assert fit["R2"] >= 0.98, f"R2 {fit['R2']:.3f} < 0.98"
    assert rel_gap <= rel_tol_vs_KL, (
        f"pente {slope_emp:.4f} vs th(KL) {slope_th:.4f} (écart rel {rel_gap:.1%})"
    )

In [48]:
# def test_M6_majority_acceptance(D=8192, seed=44):
#     pts = M6_simulate_majority_error(D=D, p=0.60, seed=seed)
#     fit = M6_fit_loglinear([(m,e,b) for (m,e,b) in pts])
#     theor_slope = -0.5 * (2*0.60 - 1)**2
#     rel_gap = abs(fit["slope"] - theor_slope) / abs(theor_slope)
#     assert fit["R2"] >= 0.98, f"R2 {fit['R2']:.3f} < 0.98"
#     assert rel_gap <= 0.15,   f"pente {fit['slope']:.4f} vs th {theor_slope:.4f}"

In [49]:
test_M6_majority_acceptance()

## M7 . Gestionnaire de segments

In [50]:
class M7_SegMgr:
    """
    Gestionnaire de segments (phrases) :
      - génère des clés K_s indépendantes (ou corrélées pour stress-test),
      - assure un reset dur à chaque frontière.
    """
    def __init__(self, seed: int, D: int):
        self.D = int(D)
        self._g = np.random.default_rng(seed)
        self.seg_idx = -1
        self._K: Optional[np.ndarray] = None
        self.onBoundary()  # crée K_0

    def curKey(self) -> np.ndarray:
        """Retourne la clé courante K_s (vue en lecture seule)."""
        assert self._K is not None
        self._K.setflags(write=False)
        return self._K

    def onBoundary(self) -> np.ndarray:
        """Ouvre un nouveau segment, génère une nouvelle clé K_{s+1} indépendante."""
        self.seg_idx += 1
        sub = int(self._g.integers(0, 2**31-1))
        K = M0_NewKey(sub, self.D)  # {-1,+1}^D int8
        K.setflags(write=False)
        self._K = K
        return self._K

    def nextKey_correlated(self, rho: float) -> np.ndarray:
        """
        (Option) Produit K_{s+1} corrélé à K_s : E[K·K'] / D ≈ rho.
        Réalisation simple: flip bit i avec proba f = (1 - rho)/2.
        """
        assert self._K is not None
        f = (1.0 - float(rho)) / 2.0
        flips = (self._g.random(self.D) < f)
        Kp = self._K.copy()
        Kp[flips] = -Kp[flips]
        Kp.setflags(write=False)
        self.seg_idx += 1
        self._K = Kp
        return self._K

def M7_SegMgr_new(seed: int, D: int) -> M7_SegMgr:
    return M7_SegMgr(seed, D)

def M7_curKey(SM: M7_SegMgr) -> np.ndarray:
    return SM.curKey()

def M7_onBoundary(SM: M7_SegMgr) -> np.ndarray:
    return SM.onBoundary()


# --- Corrections majeures apportées ---
# 1) Suppression du biais dû aux ex-aequo (ties) :
#    - Nouveau seuillage "unbiased" (tirage aléatoire ±1 si S==0)
#    - Utilisé à la fois dans M5 (n-grammes) et M6 (état de segment).
# 2) _safe_pushE robuste :
#    - Détecte dynamiquement la signature de M6_SegAcc_push (2 ou 3 args)
#    - Préserve l'absence de double-binding.
# 3) Imports manquants (inspect, numpy) + annotations.

import numpy as np
import inspect
from typing import List, Optional

# ---------- 1) Signes : strict (historique) et unbiased (corrigé) ----------

def _sign_strict_int8(x: np.ndarray) -> np.ndarray:
    """Seuillage strict: +1 si x>0, sinon -1 (int8). BIAISÉ si des zéros existent !"""
    return np.where(x > 0, 1, -1).astype(np.int8, copy=False)

def _sign_unbiased_int8(x: np.ndarray, rng: Optional[np.random.Generator] = None) -> np.ndarray:
    """
    Seuillage sans biais: +1 si x>0 ; -1 si x<0 ; pour x==0, tirage ±1 aléatoire (Rademacher).
    Cela supprime le biais d'espérance dû aux ties.
    """
    if rng is None:
        rng = np.random.default_rng(0)
    H = np.empty_like(x, dtype=np.int8)
    pos = (x > 0)
    neg = (x < 0)
    tie = ~(pos | neg)  # x == 0

    H[pos] = 1
    H[neg] = -1
    if np.any(tie):
        B = rng.integers(0, 2, size=int(tie.sum()), dtype=np.int8)
        H[tie] = (B << 1) - 1
    return H

# ---------- 2) M5 n-grammes : version strict (ancienne) et version unbiased (corrigée) ----------

def M5_ngram_cached_strict(LexEN, pi_pows: List[np.ndarray], tokens: List[str]) -> np.ndarray:
    """
    n-gramme STRICT (ancien comportement): superposition positionnelle + seuillage strict (int8).
    Attention: ce seuillage introduit un biais si l'accumulateur a des zéros.
    """
    D = LexEN.D
    acc = np.zeros(D, dtype=np.int16)
    for j, w in enumerate(reversed(tokens)):           # j=0 -> w_t
        Lw  = M4_get(LexEN, w).astype(np.int16, copy=False)
        acc += Lw[pi_pows[j]]
    return _sign_strict_int8(acc)

def M5_ngram_cached_unbiased(LexEN, pi_pows: List[np.ndarray], tokens: List[str],
                             rng: Optional[np.random.Generator] = None) -> np.ndarray:
    """
    n-gramme UNBIASED (corrigé) : superposition positionnelle + seuillage sans biais (int8).
    """
    D = LexEN.D
    acc = np.zeros(D, dtype=np.int16)
    for j, w in enumerate(reversed(tokens)):           # j=0 -> w_t
        Lw  = M4_get(LexEN, w).astype(np.int16, copy=False)
        acc += Lw[pi_pows[j]]
    return _sign_unbiased_int8(acc, rng=rng)

# ---------- 3) Push robuste : M6_SegAcc_pushE (si dispo) sinon fallback (M2->M3->M6) ----------

def _safe_pushE(S: np.ndarray,
                E_t: np.ndarray,
                Delta: int,
                K_s: np.ndarray,
                pi_pows: List[np.ndarray]) -> np.ndarray:
    """
    PUSH robuste:
      1) Si M6_SegAcc_pushE est défini et appelable, on l'emploie.
      2) Sinon, on calcule X_t = Pi^Delta(E_t), puis on choisit dynamiquement
         la bonne signature de M6_SegAcc_push pour éviter tout 'double binding':
            - push(S, X_b)      : on bind ICI (X_b = X_t ⊗ K_s)
            - push(S, X, K_s)   : on passe X_t non lié + K_s
    """
    # -- Étape 1 : tentative 'API E' si elle existe --
    fn_pushE = globals().get("M6_SegAcc_pushE", None)
    if callable(fn_pushE):
        try:
            return fn_pushE(S, E_t, Delta, K_s, pi_pows)
        except TypeError:
            # mauvaise arité: on bascule sur le fallback explicite
            pass

    # -- Étape 2 : fallback explicite (M2 -> M3 -> M6) --
    # Pi^Delta via indices précalculés
    X_t = E_t[pi_pows[Delta]].astype(np.int8, copy=False)

    # Récupérer la fonction push 'classique'
    fn_push = globals().get("M6_SegAcc_push", None)
    if not callable(fn_push):
        raise NameError("M6_SegAcc_push est introuvable (non défini dans le namespace).")

    # Déterminer la signature de push sans accéder à des symboles inexistants
    n_params = len(inspect.signature(fn_push).parameters)
    if n_params == 2:
        # push(S, X_b) : on bind ICI (sinon on rebinderait deux fois)
        X_b = M3_bind(X_t, K_s)
        return fn_push(S, X_b)
    elif n_params == 3:
        # push(S, X, K_s) : on passe X_t NON LIÉ + la clé K_s
        return fn_push(S, X_t, K_s)
    else:
        raise TypeError("Signature inattendue pour M6_SegAcc_push (attendu 2 ou 3 paramètres).")

# ---------- 4) Construction de l'état de segment : utiliser les versions 'unbiased' ----------

def M7_build_segment_state_strict(tokens: List[str],
                                  LexEN,
                                  pi_pows: List[np.ndarray],
                                  SM,
                                  n: int = 3,
                                  rng: Optional[np.random.Generator] = None) -> np.ndarray:
    """
    Chaînage 'strict' renommé mais corrigé en pratique :
      - M5 : version UNBIASED (évite le biais sur ties),
      - push via _safe_pushE,
      - majority finale UNBIASED.
    """
    if rng is None:
        rng = np.random.default_rng(0)

    D = LexEN.D
    S = M6_SegAcc_init(D)
    K = M7_curKey(SM)

    for t in range(len(tokens)):
        left = max(0, t - n + 1)
        # n-gramme sans biais
        E_t  = M5_ngram_cached_unbiased(LexEN, pi_pows, tokens[left:t+1], rng=rng)
        # push robuste (permutation relative + binding + accumulation)
        S    = _safe_pushE(S, E_t, Delta=t, K_s=K, pi_pows=pi_pows)

    # majority sans biais (ties tirés au sort)
    return _sign_unbiased_int8(S, rng=rng)

# ---------- 5) Test HA2 : inter-segments centrés et queues faibles ----------



In [51]:
def _build_H_segment(tokens: List[str],
                     LexEN,
                     pi_pows: List[np.ndarray],
                     K: np.ndarray,
                     n: int,
                     rng: np.random.Generator) -> np.ndarray:
    """Construit H^(seg) à partir d'une phrase, majorité 'unbiased'."""
    D = LexEN.D
    S = M6_SegAcc_init(D)
    for t in range(len(tokens)):
        left = max(0, t - n + 1)
        E_t  = M5_ngram_cached_unbiased(LexEN, pi_pows, tokens[left:t+1], rng=rng)
        S    = _safe_pushE(S, E_t, Delta=t, K_s=K, pi_pows=pi_pows)
    # majority unbiased (tirage ±1 si coordonnées nulles)
    return _sign_unbiased_int8(S, rng=rng)

def estimate_leak_vs_delta(D: int = 16384,
                           seed: int = 202,
                           V: int = 2000,
                           sent_len: int = 12,
                           n: int = 3,
                           deltas: List[int] = (0,1,2,4),
                           trials: int = 400) -> Tuple[np.ndarray,np.ndarray,np.ndarray]:
    """
    Estime la similarité inter-segments en fonction de Δ, avec contrôle de variance.
    - Même paires de phrases réutilisées pour tous les Δ (réduction de variance par 'couplage').
    - Retourne (deltas, means, ses).
    """
    g   = np.random.default_rng(seed)
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)

    # Pré-calcul Pi^j (indices) pour accélérer M2
    pi_pows = [np.arange(D, dtype=np.int64)]
    for _ in range(max(sent_len, n) + 2):
        pi_pows.append(pi[pi_pows[-1]])

    def rand_tok():  return f"w{int(g.integers(0, V))}"
    def rand_sent(): return [rand_tok() for _ in range(sent_len)]

    deltas = np.asarray(deltas, dtype=int)
    sims   = np.zeros((len(deltas), trials), dtype=np.float64)

    for t in range(trials):
        # -- même couple (s, s') pour toutes les valeurs de Δ : variance ↓ --
        s  = rand_sent()
        sp = rand_sent()

        # clés indépendantes
        K1 = M7_SegMgr_new(int(g.integers(1, 2**31-1)), D).curKey()
        K2 = M7_SegMgr_new(int(g.integers(1, 2**31-1)), D).curKey()

        # H^(s) (clé K1)
        H1 = _build_H_segment(s, Lex, pi_pows, K1, n=n, rng=g)

        # Pour chaque Δ : les Δ premiers tokens de s' sous K1, puis K2
        for i, Δ in enumerate(deltas):
            S2 = M6_SegAcc_init(D)
            for t2 in range(len(sp)):
                left = max(0, t2 - n + 1)
                E_t  = M5_ngram_cached_unbiased(Lex, pi_pows, sp[left:t2+1], rng=g)
                K    = K1 if t2 < Δ else K2
                S2   = _safe_pushE(S2, E_t, Delta=t2, K_s=K, pi_pows=pi_pows)
            H2 = _sign_unbiased_int8(S2, rng=g)
            sims[i, t] = M1_sim(H1.astype(np.int8), H2.astype(np.int8))

    means = sims.mean(axis=1)
    ses   = sims.std(axis=1, ddof=1) / np.sqrt(trials)
    return deltas, means, ses

# Exemple d'usage (ira plus vite en diminuant 'trials' pour un test rapide) :
# deltas, means, ses = estimate_leak_vs_delta(trials=200)
# print(list(zip(deltas, means, ses)))
# test_M7_leak_vs_delta_monotone(trials=400)

In [52]:
def test_M7_reset_vs_intra(D: int = 16384,
                           seed: int = 101,
                           V: int = 2000,
                           sent_len: int = 10,
                           n: int = 3,
                           trials: int = 300) -> None:
    """
    Deux phrases indépendantes, chacune encodée avec son propre gestionnaire (clés i.i.d.).
    Objectif HA2: similarités inter-segments centrées (≈0) et queue |sim|>0.05 faible.
    """
    g   = np.random.default_rng(seed)
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)

    # Pré-calcul Pi^j (indices) jusqu'au Δ max utile
    pi_pows = [np.arange(D, dtype=np.int64)]
    for _ in range(max(sent_len, n) + 2):
        pi_pows.append(pi[pi_pows[-1]])

    def rand_tok():  return f"w{int(g.integers(0, V))}"
    def rand_sent(): return [rand_tok() for _ in range(sent_len)]

    sims = []
    for _ in range(trials):
        SM1 = M7_SegMgr_new(int(g.integers(1, 2**31-1)), D)
        SM2 = M7_SegMgr_new(int(g.integers(1, 2**31-1)), D)
        s, sp = rand_sent(), rand_sent()

        H1 = M7_build_segment_state_strict(s,  Lex, pi_pows, SM1, n=n, rng=g)
        H2 = M7_build_segment_state_strict(sp, Lex, pi_pows, SM2, n=n, rng=g)

        sims.append(M1_sim(H1.astype(np.int8), H2.astype(np.int8)))

    sims = np.asarray(sims, dtype=np.float64)
    mean = float(sims.mean())
    tail = float((np.abs(sims) > 0.05).mean())

    # Acceptation HA2 (centrage ≈ 0 ; queues subgaussiennes)
    assert abs(mean) <= 1.5e-2, f"mean inter = {mean:.4g} trop grand"
    assert tail   <= 0.02,      f"queue empirique trop élevée: {tail:.3%}"

def test_M7_leak_vs_delta_monotone(D: int = 16384,
                                   seed: int = 202,
                                   V: int = 2000,
                                   sent_len: int = 12,
                                   n: int = 3,
                                   deltas: List[int] = (0,1,2,4),
                                   trials: int = 400) -> None:
    """
    Test 'monotone en espérance' : on autorise un petit chevauchement dû à l'incertitude.
    Condition : mean[Δ] <= mean[Δ+1] + 2*(SE_Δ + SE_{Δ+1})
    """
    deltas, means, ses = estimate_leak_vs_delta(D, seed, V, sent_len, n, deltas, trials)
    # Vérification monotone tolérante
    ok = True
    msgs = []
    for i in range(len(deltas) - 1):
        lhs = means[i]
        rhs = means[i+1] + 2.0 * (ses[i] + ses[i+1])
        if not (lhs <= rhs):
            ok = False
            msgs.append(f"Δ={deltas[i]} -> {deltas[i+1]} : {lhs:.4f} > {rhs:.4f} (tol.)")
    assert ok, "Non-monotonie au-delà de la tolérance statistique:\n" + "\n".join(msgs)


def test_M7_correlated_keys_bias(D=16384, seed=303, V=2000, sent_len=10):
    g = np.random.default_rng(seed)
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)
    pi_pows = [np.arange(D, dtype=np.int64)]
    for j in range(1, sent_len+5):
        pi_pows.append(pi[pi_pows[-1]])
    SM = M7_SegMgr_new(seed+3, D)

    rhos = [0.0, 0.05, 0.1]
    means = []
    for rho in rhos:
        sims = []
        for _ in range(40):
            s = [f"w{int(g.integers(0,V))}" for _ in range(sent_len)]
            H1 = M7_build_segment_state_strict(s, Lex, pi_pows, SM)
            # Clé suivante corrélée à la précédente
            SM.nextKey_correlated(rho)
            sp = [f"w{int(g.integers(0,V))}" for _ in range(sent_len)]
            H2 = M7_build_segment_state_strict(sp, Lex, pi_pows, SM)
            sims.append(float(np.dot(H1.astype(np.int32), H2.astype(np.int32)) / D))
        means.append(float(np.mean(sims)))
        # reset propre
        M7_onBoundary(SM)
    # Biais croissant avec rho
    assert means[0] <= means[1] + 1e-3 <= means[2] + 2e-3, f"{means}"

In [53]:
test_M7_reset_vs_intra()
test_M7_leak_vs_delta_monotone()
test_M7_correlated_keys_bias()

## M8 . Chaîne ENC (construction empilée)

In [54]:
def M8_ENC(tokens: Iterable[str],
           pi: np.ndarray,
           n: int,
           LexEN: Any,
           D: int,
           segmgr: Optional[Any] = None,
           acc_S: Optional[np.ndarray] = None,
           return_bound: bool = False,
           pi_pows: Optional[List[np.ndarray]] = None,
           majority_mode: str = "unbiased",
           m5_variant: str = "auto"
           ) -> Tuple[List[np.ndarray], List[np.ndarray], np.ndarray, np.ndarray] | \
                  Tuple[List[np.ndarray], List[np.ndarray], List[np.ndarray], np.ndarray, np.ndarray]:
    # --- garde-fous ---
    assert isinstance(D, int) and D > 0
    assert isinstance(n, int) and n >= 1
    assert isinstance(pi, np.ndarray) and pi.ndim == 1 and len(pi) == D

    # --- gestionnaire / clé ---
    if segmgr is None:
        seg_seed = int(LexEN.rng.integers(1, 2**31 - 1))
        segmgr = M7_SegMgr_new(seed=seg_seed, D=D)
    K_s = M7_curKey(segmgr)  # (D,) int8

    # --- accumulateur ---
    S = acc_S if acc_S is not None else M6_SegAcc_init(D)

    # --- pi^j (indices) minimal si absent ---
    if pi_pows is None:
        pi_pows = [np.arange(D, dtype=np.int64)]
        for _ in range(n + 2):
            pi_pows.append(pi[pi_pows[-1]])

    # --- seuillage ---
    rng = getattr(LexEN, "rng", np.random.default_rng(0))
    def _sign_unbiased_int8(x: np.ndarray, rng: np.random.Generator) -> np.ndarray:
        gt = (x > 0); lt = (x < 0); eq = ~(gt | lt)
        out = np.empty_like(x, dtype=np.int8)
        out[gt] = 1; out[lt] = -1
        if np.any(eq):
            toss = (rng.integers(0, 2, size=int(eq.sum())) * 2 - 1).astype(np.int8)
            out[eq] = toss
        return out
    def _sign_strict_int8(x: np.ndarray) -> np.ndarray:
        return np.where(x > 0, 1, -1).astype(np.int8, copy=False)

    # --- choix M5 ---
    def _pick_m5(window: Sequence[str]) -> np.ndarray:
        if m5_variant in {"unbiased","auto"}:
            fn_u = globals().get("M5_ngram_cached_unbiased", None)
            if callable(fn_u):
                return fn_u(LexEN, pi_pows, list(window), rng=rng)
            if m5_variant == "unbiased":
                raise NameError("M5_ngram_cached_unbiased indisponible.")
        if m5_variant in {"strict","auto"}:
            fn_s = globals().get("M5_ngram_cached_strict", None)
            if callable(fn_s):
                return fn_s(LexEN, pi_pows, list(window))
            if m5_variant == "strict":
                raise NameError("M5_ngram_cached_strict indisponible.")
        return M5_ngram(LexEN, pi, list(window))

    # --- push robuste ---
    def _safe_pushE(S_: np.ndarray,
                    E_t: np.ndarray,
                    Delta: int,
                    K_s_: np.ndarray) -> np.ndarray:
        fn_pushE = globals().get("M6_SegAcc_pushE", None)
        if callable(fn_pushE):
            try:
                return fn_pushE(S_, E_t, Delta, K_s_, pi_pows)
            except TypeError:
                pass
        # fallback explicite
        idx = pi_pows[Delta] if Delta < len(pi_pows) else np.arange(D, dtype=np.int64)
        X_t = E_t[idx].astype(np.int8, copy=False)
        X_b = M3_bind(X_t, K_s_)
        fn_push = globals().get("M6_SegAcc_push", None)
        if not callable(fn_push):
            raise NameError("M6_SegAcc_push indisponible.")
        import inspect
        n_params = len(inspect.signature(fn_push).parameters)
        if n_params == 2:
            return fn_push(S_, X_b)
        elif n_params == 3:
            return fn_push(S_, X_t, K_s_)
        else:
            raise TypeError("Signature inattendue pour M6_SegAcc_push (2 ou 3 paramètres attendus).")

    # --- sorties ---
    E_seq: List[np.ndarray] = []
    X_seq: List[np.ndarray] = []
    Xb_seq: Optional[List[np.ndarray]] = [] if return_bound else None

    # --- encodage glissant ---
    tok_list: List[str] = list(tokens)
    for t in range(len(tok_list)):
        left   = max(0, t - n + 1)
        window = tok_list[left : t + 1]
        E_t    = _pick_m5(window)  # (D,) int8
        Delta  = t - left
        idx    = pi_pows[Delta] if Delta < len(pi_pows) else np.arange(D, dtype=np.int64)
        X_t    = E_t[idx].astype(np.int8, copy=False)
        # *** Correctif: passer le bon nom de paramètre 'K_s_' ***
        S      = _safe_pushE(S, E_t, Delta=Delta, K_s_=K_s)
        if return_bound:
            Xb = M3_bind(X_t, K_s)
            Xb_seq.append(Xb)
        E_seq.append(E_t)
        X_seq.append(X_t)

    # seuillage final
    if majority_mode == "unbiased":
        H_s = _sign_unbiased_int8(S, rng=rng)
    elif majority_mode == "strict":
        H_s = _sign_strict_int8(S)
    else:
        raise ValueError("majority_mode ∈ {'unbiased','strict'}")

    if return_bound:
        return E_seq, X_seq, Xb_seq, S, H_s
    return E_seq, X_seq, S, H_s

In [55]:
def _build_pi_pows_with_neg(pi: np.ndarray, max_abs_k: int) -> dict[int, np.ndarray]:
    """
    Pré-calcul des puissances de permutation Pi^k pour k ∈ [-max_abs_k, ..., +max_abs_k].
    - pi: permutation de [0..D-1] (np.ndarray (D,) d'ints)
    - Retour: dict k -> indices (np.ndarray (D,))
    """
    D = pi.size
    # Puissances positives
    pows_pos = [np.arange(D, dtype=np.int64)]
    for _ in range(max_abs_k):
        pows_pos.append(pi[pows_pos[-1]])
    # Inverse pour puissances négatives
    pi_inv = np.empty_like(pi)
    pi_inv[pi] = np.arange(D, dtype=np.int64)
    pows_neg = [np.arange(D, dtype=np.int64)]
    for _ in range(max_abs_k):
        pows_neg.append(pi_inv[pows_neg[-1]])
    # Fusion dans un dict
    out: dict[int, np.ndarray] = {}
    for k in range(0, max_abs_k+1):
        out[ k] = pows_pos[k]
        out[-k] = pows_neg[k]
    return out

def test_M8_integration(seed: int = 202, D: int = 16384, n: int = 3) -> None:
    """
    Corrigé théorique & pratique.
    - Théorie: pour des décalages Δ_a, Δ_b (position relative), 
      <Pi^{Δ_a}E_a, Pi^{Δ_b}E_b> = <E_a, Pi^{Δ_b-Δ_a}E_b>.
      Donc on NE DOIT PAS comparer Gram(E) à Gram(X) directement si Δ diffèrent.
      On doit comparer Gram(X) à un Gram(E) 'aligné' par la différence de décalages.
    - Pratique: on calcule G_X puis G_E^align avec Pi^{Δ_b-Δ_a}, et on vérifie l'égalité bit-exacte.
    """
    rng = np.random.default_rng(seed)
    # dépendances
    Lex = M4_LexEN_new(seed+1, D)
    pi  = M2_plan_perm(D, seed+2)

    # Deux segments jouets
    sent1 = ["i","love","cats","very","much"]
    sent2 = ["we","like","music"]

    # Encode chaque segment séparément (on a seulement besoin des séquences {E_t} et {X_t})
    n_local = n
    E1, X1, S1, H1 = M8_ENC(sent1, pi, n_local, Lex, D)   # mode standard
    E2, X2, S2, H2 = M8_ENC(sent2, pi, n_local, Lex, D)

    # ---------- (i) Isométrie intra-segment avec alignement correct ----------
    # Définir les décalages Δ_t = t - left, où left = max(0, t-n+1)
    def deltas_for_len(T: int, n_: int) -> list[int]:
        Δ = []
        for t in range(T):
            left = max(0, t - n_ + 1)
            Δ.append(t - left)
        return Δ

    Δ1 = deltas_for_len(len(E1), n_local)  # ∈ {0,1,...,n-1} (saturation à n-1)
    Δ2 = deltas_for_len(len(E2), n_local)

    # Pré-calcul Pi^k pour k ∈ [-(n-1), ..., +(n-1)]
    pows = _build_pi_pows_with_neg(pi, max_abs_k=n_local-1)

    # Empilement en matrices (len x D) en int16 pour produits stables
    A1 = np.stack(X1).astype(np.int16, copy=False)  # (T1, D)
    B1 = np.stack(E1).astype(np.int16, copy=False)  # (T1, D)

    # Gram(X1)
    G_X1 = (A1 @ A1.T) / D  # (T1,T1) float64

    # Gram(E1) aligné: [a,b] = <E_a, Pi^{Δ_b-Δ_a} E_b> / D
    T1 = len(E1)
    G_E1_aligned = np.empty((T1, T1), dtype=np.float64)
    for a in range(T1):
        Ea = B1[a]
        for b in range(T1):
            k = Δ1[b] - Δ1[a]            # différence de décalages
            Eb_shift = B1[b, pows[k]]    # Pi^{k} E_b
            G_E1_aligned[a, b] = float(np.dot(Ea, Eb_shift) / D)

    # Tolérance: égalité exacte attendue (arithmétique entière / D), donc 0.0
    tol_iso = 0.0
    max_abs_diff_1 = float(np.max(np.abs(G_E1_aligned - G_X1)))
    assert max_abs_diff_1 <= tol_iso + 1e-12, (
        f"Isométrie violée (segment 1): max|Δ|={max_abs_diff_1:.3g} > {tol_iso}"
    )

    # Répéter pour le 2e segment (symétrie du raisonnement)
    A2 = np.stack(X2).astype(np.int16, copy=False)
    B2 = np.stack(E2).astype(np.int16, copy=False)
    G_X2 = (A2 @ A2.T) / D
    T2 = len(E2)
    G_E2_aligned = np.empty((T2, T2), dtype=np.float64)
    for a in range(T2):
        Ea = B2[a]
        for b in range(T2):
            k = Δ2[b] - Δ2[a]
            Eb_shift = B2[b, pows[k]]
            G_E2_aligned[a, b] = float(np.dot(Ea, Eb_shift) / D)

    max_abs_diff_2 = float(np.max(np.abs(G_E2_aligned - G_X2)))
    assert max_abs_diff_2 <= tol_iso + 1e-12, (
        f"Isométrie violée (segment 2): max|Δ|={max_abs_diff_2:.3g} > {tol_iso}"
    )

    # ---------- (ii) Tests de bords utiles ----------
    # sim(X_t, X_t) == 1 et dH == 0, idem pour E_t
    diag_X1 = np.diag(G_X1)
    diag_E1 = np.diag((B1 @ B1.T) / D)
    assert np.allclose(diag_X1, 1.0), "norme de X_t non préservée"
    assert np.allclose(diag_E1, 1.0), "norme de E_t non préservée"

    # ---------- (iii) Commentaire ----------
    # L'ancien test comparait Gram(E) à Gram(X) sans aligner les différences de décalages.
    # Or, si Δ_a != Δ_b, on n'a pas <Pi^{Δ_a}E_a, Pi^{Δ_b}E_b> = <E_a, E_b>, 
    # mais bien <E_a, Pi^{Δ_b-Δ_a}E_b>. Le présent test restitue cette équivariance,
    # rendant l'isométrie exacte (tolérance 0).

In [56]:
test_M8_integration()

# Encoder HDC (application concrète EN→FR sur OPUS)

In [57]:
def opus_load_subset(name: str = "opus_books", config: str = "en-fr",
                     split: str = "train", N: int = 10_000,
                     seed: int = 123) -> Tuple[List[str], List[str]]:
    """
    Charge un sous-ensemble de N paires (EN, FR) depuis huggingface/datasets.
    Ne duplique aucune primitive HDC.
    """
    ds = load_dataset(name, config, split=split)
    ds = ds.shuffle(seed=seed).select(range(min(N, len(ds))))
    ens = [ex["translation"]["en"] for ex in ds]
    frs = [ex["translation"]["fr"] for ex in ds]
    return ens, frs

In [58]:
def tokenize_en(s: str) -> List[str]:
    # cohérent avec _canon_token(w) de M4 : lower+strip, split espaces
    return [w.strip().lower() for w in s.split() if w.strip()]

def build_vocab_EN(ens: Iterable[str], V: int = 5_000) -> set:
    cnt = Counter()
    for s in ens:
        cnt.update(tokenize_en(s))
    # top-V
    return set(w for w, _ in cnt.most_common(V))

def sentence_to_tokens_EN(s: str, vocab: set) -> List[str]:
    toks = tokenize_en(s)
    # on garde tous les tokens; M4 gère OOV via pool si demandé
    return toks

In [59]:
# --- utilitaires dtypes ---
def _as_int16(x: np.ndarray) -> np.ndarray:
    return np.asarray(x, dtype=np.int16)

def _as_int8_pm1(x: np.ndarray) -> np.ndarray:
    x8 = np.asarray(x, dtype=np.int8)
    # (option) vérifs: assert np.all((x8 == -1) | (x8 == 1))
    return x8

# --- adaptateur robuste vers M6 (avec fallback sans dépendance externe) ---
def _safe_push(S: np.ndarray,
               X_t: np.ndarray,      # (D,) int8  : vecteur positionné (non lié)
               Xb_t: np.ndarray,     # (D,) int8  : X_t bindé avec K_s
               K_s: np.ndarray) -> np.ndarray:
    """
    Pousse un pas d'accumulation dans S en couvrant les deux cas:
      - \textbf{M6 présent}: on route vers \texttt{M6_SegAcc_push} si défini
        (signature (S, X_b) \ou\ (S, X, K_s)).
      - \textbf{M6 absent}: \emph{fallback} local \textbf{sans double-binding}:
        S <- S + Xb_t, avec accumulation en int16 et retour int16.
    """
    # 1) Si la primitive M6 existe, on s'y raccorde proprement.
    fn_push = globals().get("M6_SegAcc_push", None)
    if callable(fn_push):
        n_params = len(inspect.signature(fn_push).parameters)
        if n_params == 2:      # (S, X_b)
            return fn_push(S, _as_int8_pm1(Xb_t))
        if n_params == 3:      # (S, X, K_s)
            return fn_push(S, _as_int8_pm1(X_t), _as_int8_pm1(K_s))
        raise TypeError("Signature inattendue pour M6_SegAcc_push (attendu 2 ou 3 paramètres).")

    # 2) Fallback local (aucune API M6 de push disponible):
    #    on accumule \emph{uniquement} le vecteur déjà bindé (évite double-binding).
    S16  = _as_int16(S)
    Xb16 = _as_int16(Xb_t)
    S16 += Xb16
    return S16

def enc_sentence_ENC(sentence_tokens: List[str],
                     n: int,
                     pi: np.ndarray,           # plan M2 (permutation de [0..D-1])
                     LexEN: Any,               # instance M4_LexEN (.D, .get)
                     D: int,
                     seg_seed: int) -> Dict[str, Any]:
    """
    Applique la chaîne ENC à une phrase EN (un segment) en respectant:
      - position RELATIVE (Δ = t - left),
      - pas de double-binding (push robuste),
      - dtypes: int8 pour hypervecteurs, int16 pour l'accumulateur.

    Retour: dict(E_seq, X_seq, S, H, len, seg_seed)
    """
    # --- gestionnaire de segment (M7) + accumulateur (M6) ---
    SegMgr = M7_SegMgr_new(seed=seg_seed, D=D)
    K_s    = M7_curKey(SegMgr)                    # (D,) int8 (readonly)
    S      = M6_SegAcc_init(D)                    # (D,) int16

    E_seq: List[np.ndarray] = []
    X_seq: List[np.ndarray] = []

    toks = list(sentence_tokens)
    for t in range(len(toks)):
        # fenêtre n-grammes relative au début courant
        left   = max(0, t - n + 1)
        window = toks[left : t + 1]

        # 1) n-gramme positionnel (M5) : E_t ∈ {-1,+1}^D (int8)
        E_t = M5_ngram(LexEN, pi, window)

        # 2) permutation RELATIVE (M2) : Δ = t - left ∈ {0,...,n-1}
        Delta = t - left
        X_t   = M2_perm_pow(E_t, pi, Delta)       # (D,) int8

        # 3) binding de tranche (M3)
        Xb_t  = M3_bind(X_t, K_s)                 # (D,) int8

        # 4) accumulation (M6) via adaptateur (ou fallback local)
        S     = _safe_push(S, X_t, Xb_t, K_s)     # (D,) int16

        # traces
        E_seq.append(E_t); X_seq.append(X_t)

    # 5) seuillage majorité (M6)
    H_s = M6_SegAcc_sign(S).astype(np.int8, copy=False)

    return {
        "E_seq":   E_seq,
        "X_seq":   X_seq,
        "S":       S,
        "H":       H_s,
        "len":     len(toks),
        "seg_seed": seg_seed,
    }

<>:18: SyntaxWarning: invalid escape sequence '\o'
<>:18: SyntaxWarning: invalid escape sequence '\o'
/var/folders/zg/w26r16bd5c11s4xr1f6lb4mh0000gn/T/ipykernel_38343/2942300705.py:18: SyntaxWarning: invalid escape sequence '\o'
  (signature (S, X_b) \ou\ (S, X, K_s)).


In [60]:
# ===========================
# Logging & helpers (communs)
# ===========================
import logging, inspect
from typing import List, Dict, Any, Tuple, Iterable, Sequence, Optional
import numpy as np

log = logging.getLogger("ENC.pipeline")
if not log.handlers:
    logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def _sign_strict_int8(x: np.ndarray) -> np.ndarray:
    """Retourne un vecteur int8 dans {-1,+1} avec règle stricte (+1 si x>0, sinon -1)."""
    return np.where(x > 0, 1, -1).astype(np.int8, copy=False)

# ===========================================================
# Adaptateurs robustes pour l'accumulation (M6) avec fallback
# ===========================================================

_warned_local_accum = False  # pour ne logger le fallback qu'une seule fois

def _safe_push(S: np.ndarray,
               X_t: np.ndarray,
               X_b: np.ndarray,
               K_s: np.ndarray) -> np.ndarray:
    """
    Adaptateur robuste pour l'accumulation M6.
    Ordre de préférence:
      1) M6_SegAcc_push(S, X_b)                [signature 2 args]
      2) M6_SegAcc_push(S, X_t, K_s)           [signature 3 args]
      3) Fallback local: S += X_b.astype(int16) (log d'avertissement unique)
    Hyp.: S est int16 (shape (D,)); X_t, X_b, K_s sont int8 dans {-1,+1}.
    """
    global _warned_local_accum

    fn_push = globals().get("M6_SegAcc_push", None)
    if callable(fn_push):
        n_params = len(inspect.signature(fn_push).parameters)
        if n_params == 2:
            # signature (S, X) -> on passe la version déjà bindée
            return fn_push(S, X_b)
        elif n_params == 3:
            # signature (S, X, K) -> on passe X_t non bindé + K_s
            return fn_push(S, X_t, K_s)
        else:
            log.warning("Signature inattendue pour M6_SegAcc_push (attendu 2 ou 3 paramètres). "
                        "Fallback local déclenché.")

    # --- Fallback local: accumulation explicite coordonnée ---
    if not _warned_local_accum:
        log.warning("M6_SegAcc_push introuvable: utilisation d'un accumulateur local "
                    "S += X_b (int16). Vérifiez l'import de M6.")
        _warned_local_accum = True

    # sécurité dtype
    if S.dtype != np.int16:
        S = S.astype(np.int16, copy=False)
    S += X_b.astype(np.int16, copy=False)
    return S

# ===========================
# Encodage corpus (avec logs)
# ===========================

def encode_corpus_ENC(ens: List[str],
                      LexEN,
                      pi: np.ndarray,
                      D: int, n: int,
                      seg_seed0: int = 10_001,
                      log_every: int = 1_000) -> List[Dict[str, Any]]:
    """
    Encode un corpus de phrases anglaises (liste 'ens') via la chaîne ENC.
    - Ajoute des logs de progression toutes les 'log_every' phrases.
    - Réutilise strictement les primitives M2..M7 (pas de redondance).
    """
    out: List[Dict[str, Any]] = []
    g = np.random.default_rng(seg_seed0)
    total = len(ens)
    log.info("ENC: démarrage encodage %d phrases (n=%d, D=%d)...", total, n, D)

    for i, s in enumerate(ens, 1):
        toks = sentence_to_tokens_EN(s, vocab=set())  # le vocab/BPE a déjà été traité côté M4
        seg_seed = int(g.integers(1, 2**31-1))
        out.append(enc_sentence_ENC(toks, n, pi, LexEN, D, seg_seed))

        if i % log_every == 0 or i == total:
            log.info("ENC: %d/%d phrases encodées (%.1f%%).", i, total, 100.0 * i / total)

    log.info("ENC: encodage terminé.")
    return out

# ===========================================
# Encodage d'une phrase (segment) avec traces
# ===========================================

def enc_sentence_ENC(sentence_tokens: List[str],
                     n: int,
                     pi: np.ndarray,           # plan M2
                     LexEN,                    # instance M4_LexEN
                     D: int,
                     seg_seed: int) -> Dict[str, Any]:
    """
    Applique la chaîne ENC à une phrase EN (un segment).
    Retourne E_seq, X_seq, S, H et des méta-données utiles (longueur, seed, etc.).
    """
    # 0) gestionnaire de segment + accumulateur
    SegMgr = M7_SegMgr_new(seed=seg_seed, D=D)  # M7
    K_s    = M7_curKey(SegMgr)                  # (D,) int8 readonly
    S      = M6_SegAcc_init(D)                 # (D,) int16

    E_seq: List[np.ndarray] = []
    X_seq: List[np.ndarray] = []

    toks = sentence_tokens
    for t in range(len(toks)):
        # 1) fenêtre n-grammes (positionnelle) -> E_t (M5)
        left   = max(0, t - n + 1)
        window = toks[left:t+1]
        E_t    = M5_ngram(LexEN, pi, window).astype(np.int8, copy=False)   # (D,) int8

        # 2) position relative -> X_t = Pi^{t-left}(E_t) (M2)
        Delta  = t - left
        X_t    = M2_perm_pow(E_t, pi, Delta).astype(np.int8, copy=False)   # (D,) int8

        # 3) binding -> Xb_t (M3)
        Xb_t   = M3_bind(X_t, K_s).astype(np.int8, copy=False)             # (D,) int8

        # 4) accumulation (M6) via adaptateur (ou fallback local)
        S      = _safe_push(S, X_t, Xb_t, K_s)                             # (D,) int16

        # 5) traces
        E_seq.append(E_t); X_seq.append(X_t)

    # 6) seuillage final (majorité). On préfère la règle stricte (évite biais sur 0)
    #    si M6_SegAcc_sign n'est pas présent, on applique la version stricte locale.
    if "M6_SegAcc_sign" in globals() and callable(globals()["M6_SegAcc_sign"]):
        H_s = globals()["M6_SegAcc_sign"](S)
        if H_s.dtype != np.int8:
            H_s = H_s.astype(np.int8, copy=False)
    else:
        log.warning("M6_SegAcc_sign introuvable: seuillage STRICT local appliqué.")
        H_s = _sign_strict_int8(S)

    return {"E_seq": E_seq, "X_seq": X_seq, "S": S, "H": H_s,
            "len": len(toks), "seg_seed": seg_seed}

# =====================================================
# Mesures de similarité intra/inter n-grammes (avec log)
# =====================================================

def intra_inter_ngram_sims(E_seq_list: List[List[np.ndarray]], D: int) -> Tuple[float, float]:
    """
    Intra: moyenne des similarités entre n-grammes consécutifs d'une même phrase.
    Inter: moyenne des |similarités| entre premiers n-grammes de phrases consécutives.
    """
    log.info("Calcul des similarités intra/inter n-grammes...")
    s_intra_vals: List[float] = []
    for E_seq in E_seq_list:
        for a in range(len(E_seq) - 1):
            s_intra_vals.append(M1_sim(E_seq[a], E_seq[a + 1]))
    s_intra = float(np.mean(s_intra_vals)) if s_intra_vals else 0.0

    s_inter_vals: List[float] = []
    for i in range(len(E_seq_list) - 1):
        Ei = E_seq_list[i]
        Ej = E_seq_list[i + 1]
        if Ei and Ej:
            s_inter_vals.append(abs(M1_sim(Ei[0], Ej[0])))
    s_inter = float(np.mean(s_inter_vals)) if s_inter_vals else 0.0

    log.info("Similarités n-grammes -> intra=%.4f, inter(abs)=%.4f", s_intra, s_inter)
    return s_intra, s_inter

# ======================================
# Similarité inter-segments H (avec logs)
# ======================================

def inter_segment_similarity(H_list: List[np.ndarray]) -> float:
    """
    |sim(H^{(s)}, H^{(s+1)})| moyen sur segments consécutifs (≈ 0 à grande D).
    """
    log.info("Calcul des similarités inter-segments H...")
    vals: List[float] = []
    for i in range(len(H_list) - 1):
        vals.append(abs(M1_sim(H_list[i].astype(np.int8), H_list[i + 1].astype(np.int8))))
    mean_inter = float(np.mean(vals)) if vals else 0.0
    log.info("Similarité inter-segments (abs mean) = %.5f", mean_inter)
    return mean_inter

# =========================================================
# Courbes d'erreur de majorité (strict, robuste, avec logs)
# =========================================================

# -- Adaptateur push 'classique' (M6_SegAcc_push) si pushE indispo --
def _safe_push_with_X(S: np.ndarray, X_t: np.ndarray, K_s: np.ndarray) -> np.ndarray:
    fn_push = globals().get("M6_SegAcc_push", None)
    if not callable(fn_push):
        raise NameError("Ni M6_SegAcc_pushE ni M6_SegAcc_push n'est disponible dans le namespace.")
    n_params = len(inspect.signature(fn_push).parameters)
    if n_params == 2:
        # signature (S, X_b): on fournit déjà le binding
        X_b = M3_bind(X_t, K_s)
        return fn_push(S, X_b)
    elif n_params == 3:
        # signature (S, X, K_s): on laisse M6 faire le binding
        return fn_push(S, X_t, K_s)
    else:
        raise TypeError("Signature inattendue pour M6_SegAcc_push (attendu 2 ou 3 paramètres).")


def majority_error_curve(E_seq_list: List[List[np.ndarray]],
                         pi: np.ndarray,
                         D: int,
                         eta_list: Tuple[float, ...] = (0.0, 0.05, 0.10),
                         seed_noise: int = 303) -> Dict[float, List[Tuple[int, float]]]:
    """
    Courbes d'erreur de majorité avec *même clé K_s* pour clean & noisy,
    et *même pipeline* (toujours via _safe_push_with_X).
    Retourne: {eta: [(m, err_emp_moyenne), ...]}.
    """
    g = np.random.default_rng(seed_noise)

    def _flip_noise_pm1(X: np.ndarray, eta: float) -> np.ndarray:
        if eta <= 0.0:
            return X.astype(np.int8, copy=False)
        mask  = (g.random(X.shape[0]) < eta).astype(np.int8)
        flips = (1 - 2*mask).astype(np.int8)   # 0->+1, 1->-1
        return (X.astype(np.int8, copy=False) * flips).astype(np.int8, copy=False)

    per_eta: Dict[float, List[Tuple[int, float]]] = {eta: [] for eta in eta_list}

    for E_seq in E_seq_list:
        m = len(E_seq)
        if m == 0:
            continue

        # --- 1) même clé pour clean & noisy ---
        SM = M7_SegMgr_new(seed=int(g.integers(1, 2**31-1)), D=D)
        K  = M7_curKey(SM)  # (D,) int8

        # --- 2) baseline clean (même chemin) ---
        S_clean = M6_SegAcc_init(D)
        for t, E_t in enumerate(E_seq):
            # position relative: X_t = Pi^t(E_t)
            X_t = M2_perm_pow(E_t, pi, t).astype(np.int8, copy=False)
            S_clean = _safe_push_with_X(S_clean, X_t, K)
        H_clean = M6_SegAcc_sign(S_clean)

        # --- 3) variantes bruitées (même clé & même chemin) ---
        for eta in eta_list:
            S_noisy = M6_SegAcc_init(D)
            for t, E_t in enumerate(E_seq):
                X_t = M2_perm_pow(E_t, pi, t).astype(np.int8, copy=False)
                X_t = _flip_noise_pm1(X_t, eta)          # seul changement côté noisy
                S_noisy = _safe_push_with_X(S_noisy, X_t, K)
            H_noisy = M6_SegAcc_sign(S_noisy)

            # fraction de bits différents
            diff = (H_clean.astype(np.int8) * H_noisy.astype(np.int8) == -1).mean()
            per_eta[eta].append((m, float(diff)))

    # Agrégation: moyenne par longueur m
    reduced: Dict[float, List[Tuple[int, float]]] = {}
    for eta, pairs in per_eta.items():
        by_m: Dict[int, List[float]] = {}
        for m, e in pairs:
            by_m.setdefault(m, []).append(e)
        reduced[eta] = sorted((m, float(np.mean(es))) for m, es in by_m.items())
    return reduced

In [61]:
# --- utilitaires locaux pour M6 si les primitives ne sont pas importées ---
def _local_segacc_init(D: int) -> np.ndarray:
    return np.zeros(D, dtype=np.int16)

def _local_bind(X_t: np.ndarray, K_s: np.ndarray) -> np.ndarray:
    # retourne X_b en int8 dans {-1,+1}
    return (X_t.astype(np.int8, copy=False) * K_s.astype(np.int8, copy=False)).astype(np.int8, copy=False)

def _local_segacc_push(S: np.ndarray, X_t: np.ndarray, K_s: np.ndarray) -> np.ndarray:
    # accumulation locale: bind + somme en int16
    if S.dtype != np.int16:
        S = S.astype(np.int16, copy=False)
    X_b = _local_bind(X_t, K_s).astype(np.int16, copy=False)
    S  += X_b
    return S

def _local_segacc_sign(S: np.ndarray) -> np.ndarray:
    # seuillage STRICT (évite biais sur 0): +1 si S>0, sinon -1
    return np.where(S > 0, 1, -1).astype(np.int8, copy=False)

# --- adaptateurs génériques vers M6 (avec fallback local si absent) ---
def _segacc_init(D: int) -> np.ndarray:
    fn = globals().get("M6_SegAcc_init", None)
    return fn(D) if callable(fn) else _local_segacc_init(D)

def _segacc_sign(S: np.ndarray) -> np.ndarray:
    fn = globals().get("M6_SegAcc_sign", None)
    return fn(S) if callable(fn) else _local_segacc_sign(S)

def _safe_push_with_X(S: np.ndarray, X_t: np.ndarray, K_s: np.ndarray) -> np.ndarray:
    """
    Essaie M6_SegAcc_push puis retombe sur le push local si absent.
    - (S, X_b)  : fournir X_b déjà bindé,
    - (S, X, K) : fournir X_t non bindé + K_s,
    - sinon     : fallback local (bind + somme int16).
    """
    fn_push = globals().get("M6_SegAcc_push", None)
    if callable(fn_push):
        try:
            n_params = len(inspect.signature(fn_push).parameters)
        except (TypeError, ValueError):
            n_params = 0
        if n_params == 2:
            return fn_push(S, _local_bind(X_t, K_s))
        if n_params == 3:
            return fn_push(S, X_t, K_s)
        # signature inattendue -> fallback local
    # pas de push M6: fallback local
    return _local_segacc_push(S, X_t, K_s)

def _safe_push_with_E(S: np.ndarray,
                      E_t: np.ndarray,
                      Delta: int,
                      K_s: np.ndarray,
                      pi_pows: List[np.ndarray],
                      pi: np.ndarray) -> np.ndarray:
    """
    Essaie M6_SegAcc_pushE(S, E_t, Delta, K_s, pi_pows); sinon calcule X_t=Pi^Delta(E_t)
    et utilise _safe_push_with_X.
    """
    fn_pushE = globals().get("M6_SegAcc_pushE", None)
    if callable(fn_pushE):
        try:
            return fn_pushE(S, E_t, Delta, K_s, pi_pows)
        except TypeError:
            # mauvaise arité -> fallback X
            pass
    # fallback: utiliser Pi^Delta côté M2 puis push classique
    X_t = M2_perm_pow(E_t, pi, Delta)
    return _safe_push_with_X(S, X_t, K_s)

# ------------------------------------------------------------
# Remplacement sécurisé de majority_error_curve (plus d'erreur)
# ------------------------------------------------------------
def majority_error_curve(E_seq_list: List[List[np.ndarray]],
                         pi: np.ndarray,
                         D: int,
                         eta_list: Tuple[float, ...] = (0.0, 0.05, 0.10),
                         seed_noise: int = 303) -> Dict[float, List[Tuple[int, float]]]:
    """
    Rejoue l'accumulation 'clean' puis 'noisy' (flips sur X_t) pour chaque séquence E_t.
    Compatible avec:
      - M6_SegAcc_pushE + M6_SegAcc_sign,
      - M6_SegAcc_push (2 ou 3 args) + M6_SegAcc_sign,
      - ou *aucune* primitive M6 (fallback local sans dépendance).
    """
    g = np.random.default_rng(seed_noise)

    use_pushE = callable(globals().get("M6_SegAcc_pushE", None))
    pi_pows = None
    if use_pushE:
        max_m = max((len(seq) for seq in E_seq_list), default=0)
        pi_pows = [np.arange(D, dtype=np.int64)]
        for _ in range(max_m + 2):
            pi_pows.append(pi[pi_pows[-1]])

    def _flip_noise_pm1(X: np.ndarray, eta: float) -> np.ndarray:
        if eta <= 0.0:
            return X.astype(np.int8, copy=False)
        mask  = (g.random(X.shape[0]) < eta).astype(np.int8)
        flips = (1 - 2*mask).astype(np.int8)  # 0->+1, 1->-1
        return (X.astype(np.int8, copy=False) * flips).astype(np.int8, copy=False)

    per_eta: Dict[float, List[Tuple[int, float]]] = {eta: [] for eta in eta_list}

    for E_seq in E_seq_list:
        m = len(E_seq)
        if m == 0:
            continue

        # même clé K_s pour clean/noisy (SegMgr)
        SM = M7_SegMgr_new(seed=7_777, D=D)
        K  = M7_curKey(SM)

        # --- Clean ---
        S_clean = _segacc_init(D)
        for t, E_t in enumerate(E_seq):
            if use_pushE:
                S_clean = _safe_push_with_E(S_clean, E_t, Delta=t, K_s=K, pi_pows=pi_pows, pi=pi)
            else:
                X_t = M2_perm_pow(E_t, pi, t)
                S_clean = _safe_push_with_X(S_clean, X_t, K)
        H_clean = _segacc_sign(S_clean)

        # --- Noisy ---
        for eta in eta_list:
            S_noisy = _segacc_init(D)
            for t, E_t in enumerate(E_seq):
                X_t = M2_perm_pow(E_t, pi, t)
                X_t = _flip_noise_pm1(X_t, eta)
                S_noisy = _safe_push_with_X(S_noisy, X_t, K)
            H_noisy = _segacc_sign(S_noisy)

            # fraction de bits différents
            diff = (H_clean.astype(np.int8) * H_noisy.astype(np.int8) == -1).mean()
            per_eta[eta].append((m, float(diff)))

    # agrégation: moyenne par longueur m
    reduced: Dict[float, List[Tuple[int, float]]] = {}
    for eta, pairs in per_eta.items():
        by_m: Dict[int, List[float]] = {}
        for m, e in pairs:
            by_m.setdefault(m, []).append(e)
        reduced[eta] = sorted((m, float(np.mean(es))) for m, es in by_m.items())
    return reduced

In [62]:
import logging

log = logging.getLogger("validate_on_opus")
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def validate_on_opus(D: int = 16_384, n: int = 3, N: int = 1000,
                     seed_lex: int = 10_123, seed_pi: int = 10_456) -> dict:
    log.info("Chargement du sous-corpus OPUS (N=%d)...", N)
    ens, frs = opus_load_subset("opus_books", "en-fr", "train", N=N, seed=2024)

    log.info("Initialisation du lexique anglais (M4) avec seed=%d et dimension D=%d...", seed_lex, D)
    Lex = M4_LexEN_new(seed_lex, D, reserve_pool=4_096)

    log.info("Construction du plan de permutation (M2) avec seed=%d...", seed_pi)
    pi  = M2_plan_perm(D, seed_pi)

    log.info("Encodage des phrases anglaises avec ENC (n=%d, seg_seed0=9001)...", n)
    encoded = encode_corpus_ENC(ens, Lex, pi, D, n, seg_seed0=9_001)

    log.info("Extraction des séquences E et signatures H...")
    E_list  = [e["E_seq"] for e in encoded]
    H_list  = [e["H"] for e in encoded]

    log.info("Calcul des similarités intra/inter n-grammes...")
    s_intra, s_inter = intra_inter_ngram_sims(E_list, D)

    log.info("Calcul des similarités inter-segments...")
    s_inter_seg = inter_segment_similarity(H_list)

    log.info("Calcul des courbes d'erreur de majorité (eta = 0, 0.05, 0.1)...")
    maj_curves = majority_error_curve(E_list, pi, D, eta_list=(0.0, 0.05, 0.1))

    log.info("Compilation des résultats...")
    results = {
        "D": D, "n": n, "N_pairs": N,
        "seed_lex": seed_lex, "seed_pi": seed_pi,
        "intra_ngram_mean_sim": s_intra,
        "inter_ngram_abs_mean_sim": s_inter,
        "inter_segment_abs_mean_sim": s_inter_seg,
        "majority_curves": maj_curves
    }

    log.info("Validation terminée avec succès.")
    return results

In [63]:
res = validate_on_opus()

2025-09-30 15:13:40,234 [INFO] Chargement du sous-corpus OPUS (N=1000)...
2025-09-30 15:13:43,229 [INFO] Initialisation du lexique anglais (M4) avec seed=10123 et dimension D=16384...
2025-09-30 15:13:43,293 [INFO] Construction du plan de permutation (M2) avec seed=10456...
2025-09-30 15:13:43,295 [INFO] Encodage des phrases anglaises avec ENC (n=3, seg_seed0=9001)...
2025-09-30 15:13:43,295 [INFO] ENC: démarrage encodage 1000 phrases (n=3, D=16384)...
2025-09-30 15:13:43,300 [WARNING] M6_SegAcc_push introuvable: utilisation d'un accumulateur local S += X_b (int16). Vérifiez l'import de M6.
2025-09-30 15:15:05,225 [INFO] ENC: 1000/1000 phrases encodées (100.0%).
2025-09-30 15:15:05,225 [INFO] ENC: encodage terminé.
2025-09-30 15:15:05,225 [INFO] Extraction des séquences E et signatures H...
2025-09-30 15:15:05,225 [INFO] Calcul des similarités intra/inter n-grammes...
2025-09-30 15:15:05,226 [INFO] Calcul des similarités intra/inter n-grammes...
2025-09-30 15:15:05,468 [INFO] Similarit

### Tester l'hypothèse ENC3 : 

In [66]:
# ===========================================================
# Courbes de majorité ENC3 "pur" : vecteur répété + flips (simulation)
# ===========================================================
import numpy as np
import math
from typing import Dict, List, Tuple

def majority_curve_repeated_vector(E_seq_list: List[List[np.ndarray]],
                                   pi: np.ndarray,
                                   D: int,
                                   eta_list: Tuple[float, ...] = (0.0, 0.05, 0.10),
                                   trials_per_m: int = 4000,
                                   seed: int = 4242) -> Dict[float, List[Tuple[int, float]]]:
    """
    Calcule des courbes d'erreur pour le vote majoritaire en condition ENC3 "pur":
      - On répète m fois le *même* hypervecteur de référence X_ref ∈ {-1,+1}^D.
      - À chaque répétition, chaque coordonnée subit un flip indépendant de probabilité eta.
      - On estime l'erreur bit par bit: err(m, eta) = P[sum_{k=1..m} Y_k <= 0], où Y_k ∈ {-1,+1},
        P(Y_k=+1) = 1-eta, P(Y_k=-1) = eta. (Ties comptées comme erreurs ➜ règle *stricte*.)
      - L'erreur ne dépend PAS de X_ref ni de D (i.i.d. par coordonnée), mais on renvoie les points
        pour un ensemble de longueurs m *observées* dans le corpus (pour rester commensurable avec
        vos longueurs de phrases).
    Paramètres
    ----------
    E_seq_list : liste des séquences d'E_t (utilisée uniquement pour produire la grille des m rencontrés).
    pi, D      : non utilisés dans ce protocole (présents pour signature homogène).
    eta_list   : niveaux de bruit à tester (flips).
    trials_per_m : nombre d'essais Monte Carlo par longueur m (plus grand ➜ variance plus faible).
    seed       : seed RNG.
    Retour
    ------
    dict: {eta: [(m, err_empirique), ...]} avec m croissants.
    """
    # Grille des longueurs m rencontrées dans le corpus (hors m=0)
    m_grid = sorted({len(seq) for seq in E_seq_list if len(seq) > 0})
    if not m_grid:
        return {eta: [] for eta in eta_list}

    g = np.random.default_rng(seed)
    out: Dict[float, List[Tuple[int, float]]] = {eta: [] for eta in eta_list}

    for eta in eta_list:
        # Rappel: p = P(Y=+1) = 1 - eta
        p = 1.0 - float(eta)
        for m in m_grid:
            # On simule 'trials_per_m' expériences indépendantes:
            # Pour chaque expérience, on tire m variables de Bernoulli(p) puis on
            # mappe {1 -> +1, 0 -> -1}, on somme et on teste (sum > 0 ? ok : erreur).
            # Avec règle stricte, ties (sum==0) sont des erreurs.
            # NB: on n'a pas besoin de dimension D ici, l'événement est par coordonnée.
            B = (g.random(size=(trials_per_m, m)) < p)    # True ~ +1, False ~ -1
            Y = np.where(B, 1, -1).astype(np.int16)       # (trials, m)
            S = Y.sum(axis=1)                              # (trials,)
            err = float((S <= 0).mean())                  # tie inclus comme erreur
            out[eta].append((m, err))

    # Tri des listes par m (pour un affichage propre)
    for eta in eta_list:
        out[eta].sort(key=lambda t: t[0])
    return out


# ===========================================================
# Adaptation: validate_on_opus utilise now majority_curve_repeated_vector
# ===========================================================
import logging

log = logging.getLogger("validate_on_opus")
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def validate_on_opus_enc3(D: int = 16_384, n: int = 3, N: int = 1000,
                     seed_lex: int = 10_123, seed_pi: int = 10_456) -> dict:
    log.info("Chargement du sous-corpus OPUS (N=%d)...", N)
    ens, frs = opus_load_subset("opus_books", "en-fr", "train", N=N, seed=2024)

    log.info("Initialisation du lexique anglais (M4) avec seed=%d et dimension D=%d...", seed_lex, D)
    Lex = M4_LexEN_new(seed_lex, D, reserve_pool=4_096)

    log.info("Construction du plan de permutation (M2) avec seed=%d...", seed_pi)
    pi  = M2_plan_perm(D, seed_pi)

    log.info("Encodage des phrases anglaises avec ENC (n=%d, seg_seed0=9001)...", n)
    encoded = encode_corpus_ENC(ens, Lex, pi, D, n, seg_seed0=9_001)

    log.info("Extraction des séquences E et signatures H...")
    E_list  = [e["E_seq"] for e in encoded]
    H_list  = [e["H"] for e in encoded]

    log.info("Calcul des similarités intra/inter n-grammes...")
    s_intra, s_inter = intra_inter_ngram_sims(E_list, D)

    log.info("Calcul des similarités inter-segments...")
    s_inter_seg = inter_segment_similarity(H_list)

    log.info("Calcul des courbes d'erreur de majorité ENC3 (vecteur répété) pour eta = 0, 0.05, 0.1...")
    # ➜ Remplace l’ancien protocole 'majority_error_curve' (pipeline réel)
    maj_curves = majority_curve_repeated_vector(E_list, pi, D, eta_list=(0.0, 0.05, 0.1),
                                                trials_per_m=4000, seed=4242)

    log.info("Compilation des résultats...")
    results = {
        "D": D, "n": n, "N_pairs": N,
        "seed_lex": seed_lex, "seed_pi": seed_pi,
        "intra_ngram_mean_sim": s_intra,
        "inter_ngram_abs_mean_sim": s_inter,
        "inter_segment_abs_mean_sim": s_inter_seg,
        "majority_curves": maj_curves
    }

    log.info("Validation terminée avec succès.")
    return results

In [67]:
res_enc3 = validate_on_opus_enc3()

2025-09-30 15:23:20,995 [INFO] Chargement du sous-corpus OPUS (N=1000)...
2025-09-30 15:23:23,189 [INFO] Initialisation du lexique anglais (M4) avec seed=10123 et dimension D=16384...
2025-09-30 15:23:23,254 [INFO] Construction du plan de permutation (M2) avec seed=10456...
2025-09-30 15:23:23,256 [INFO] Encodage des phrases anglaises avec ENC (n=3, seg_seed0=9001)...
2025-09-30 15:23:23,256 [INFO] ENC: démarrage encodage 1000 phrases (n=3, D=16384)...
2025-09-30 15:24:44,426 [INFO] ENC: 1000/1000 phrases encodées (100.0%).
2025-09-30 15:24:44,426 [INFO] ENC: encodage terminé.
2025-09-30 15:24:44,426 [INFO] Extraction des séquences E et signatures H...
2025-09-30 15:24:44,427 [INFO] Calcul des similarités intra/inter n-grammes...
2025-09-30 15:24:44,427 [INFO] Calcul des similarités intra/inter n-grammes...
2025-09-30 15:24:44,670 [INFO] Similarités n-grammes -> intra=0.0004, inter(abs)=0.0212
2025-09-30 15:24:44,670 [INFO] Calcul des similarités inter-segments...
2025-09-30 15:24:44,6

In [68]:
res_enc3

{'D': 16384,
 'n': 3,
 'N_pairs': 1000,
 'seed_lex': 10123,
 'seed_pi': 10456,
 'intra_ngram_mean_sim': 0.0003932793581665431,
 'inter_ngram_abs_mean_sim': 0.021165819139451952,
 'inter_segment_abs_mean_sim': 0.010303027637012012,
 'majority_curves': {0.0: [(1, 0.0),
   (2, 0.0),
   (3, 0.0),
   (4, 0.0),
   (5, 0.0),
   (6, 0.0),
   (7, 0.0),
   (8, 0.0),
   (9, 0.0),
   (10, 0.0),
   (11, 0.0),
   (12, 0.0),
   (13, 0.0),
   (14, 0.0),
   (15, 0.0),
   (16, 0.0),
   (17, 0.0),
   (18, 0.0),
   (19, 0.0),
   (20, 0.0),
   (21, 0.0),
   (22, 0.0),
   (23, 0.0),
   (24, 0.0),
   (25, 0.0),
   (26, 0.0),
   (27, 0.0),
   (28, 0.0),
   (29, 0.0),
   (30, 0.0),
   (31, 0.0),
   (32, 0.0),
   (33, 0.0),
   (34, 0.0),
   (35, 0.0),
   (36, 0.0),
   (37, 0.0),
   (38, 0.0),
   (39, 0.0),
   (40, 0.0),
   (41, 0.0),
   (42, 0.0),
   (43, 0.0),
   (44, 0.0),
   (45, 0.0),
   (46, 0.0),
   (47, 0.0),
   (48, 0.0),
   (49, 0.0),
   (50, 0.0),
   (51, 0.0),
   (52, 0.0),
   (53, 0.0),
   (54, 0.0)